In [ ]:
# import tensorflow as tf
# from tensorflow.keras import layers, Model
# from tensorflow.keras.metrics import BinaryAccuracy, Precision, Recall, AUC
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import balanced_accuracy_score, confusion_matrix, accuracy_score
# import os


# BATCH_SIZE = 32
# # Define the number of training epochs
# NUM_EPOCHS = 200 # You can adjust this number

# # Parameters for AMRL
# M_POS = 2.0
# M_NEG = -2.0
# LAMBDA_POS = 10.0
# LAMBDA_NEG = 1.0
# BETA_POS = 2
# BETA_NEG = 1

# from google.colab import drive
# drive.mount('/content/drive')

# # Define your Google Drive base path (adjust as needed)
# GOOGLE_DRIVE_PATH = '/content/drive/MyDrive/converted_excel_files_all/'

# results_file_path = '/content/drive/MyDrive//imbalanced_loss_comparison_summary.xlsx'

# # --- 5. Neural Network Model (TensorFlow/Keras) ---
# class SimpleMLP_TF(Model):
#     def __init__(self, input_dim):
#         super(SimpleMLP_TF, self).__init__()
#         #self.fc1 = layers.Dense(128, activation='relu')
#         self.fc1 = layers.Dense(128, activation='relu', input_shape=(input_dim,))
#         self.fc2 = layers.Dense(64, activation='relu')
#         self.fc3 = layers.Dense(1) # Output a single logit for binary classification

#     def call(self, inputs):
#         x = self.fc1(inputs)
#         x = self.fc2(x)
#         return self.fc3(x)


# # --- 2. Data Loading from Google Drive ---
# # This function is now the sole method for loading data.
# def load_data_from_drive(file_path):
#     """
#     Loads a dataset from a given file path.
#     Assumes the last column is the target and no header row in the CSV file.
#     Args:
#         file_path (str): The full path to the CSV file.
#     Returns:
#         pd.DataFrame: Loaded DataFrame.
#     """
#     if not os.path.exists(file_path):
#         print(f"Error: File not found at {file_path}.")
#         return None

#     # Read CSV assuming no header. Pandas will assign integer column names (0, 1, 2, ...)
#     print(f"Attempting to read Excel file: {file_path}")
#     try:
#         df = pd.read_excel(file_path, header=None)
#         print(f"Loaded {os.path.basename(file_path)} with shape: {df.shape}")
#     except Exception as e:
#         print(f"Error reading Excel file {file_path}: {e}")
#         # Attempt to read as CSV as a fallback
#         try:
#             df = pd.read_csv(file_path, header=None)
#             print(f"Loaded {os.path.basename(file_path)} as CSV with shape: {df.shape}")
#         except Exception as e_csv:
#             print(f"Error reading CSV file {file_path}: {e_csv}")
#             print(f"Failed to load {file_path} as both Excel and CSV.")
#             return None


#     # The last column will be identified by its index (e.g., df.columns[-1] will be the last integer index).
#     print(f"Assuming the last column '{df.columns[-1]}' is the target column (no header specified).")
#     print(f"Class distribution of the last column:\n{df.iloc[:, -1].value_counts()}")
#     return df


# # --- Loss Function Implementations for TensorFlow ---

# class AMRLLossTF(tf.keras.losses.Loss):
#   """
#   Margin Loss for Imbalanced Datasets in TensorFlow.
#   Combines Weighted Binary Cross-Entropy with an Asymmetric Margin Penalty.
#   """
#   def __init__(self, w1, w0, m1, m0, lambda1, lambda0, beta1=1, beta0=1, name="margin_loss_tf"):
#     super().__init__(name=name)
#     self.w1 = tf.constant(w1, dtype=tf.float32)
#     self.w0 = tf.constant(w0, dtype=tf.float32)
#     # These are now trainable variables!
#     self.m1 = tf.Variable(m1, dtype=tf.float32, trainable=True, name="m1")
#     self.m0 = tf.Variable(m0, dtype=tf.float32, trainable=True, name="m0")
#     self.lambda1 = tf.Variable(lambda1, dtype=tf.float32, trainable=True, name="lambda1")
#     self.lambda0 = tf.Variable(lambda0, dtype=tf.float32, trainable=True, name="lambda0")
#     self.beta1 = tf.constant(float(beta1), dtype=tf.float32)
#     self.beta0 = tf.constant(float(beta0), dtype=tf.float32)

#   def call(self, y_true, y_pred_logits):
#     y_true = tf.cast(y_true, tf.float32)
#     y_pred_logits = tf.cast(y_pred_logits, tf.float32)

#     bce_unweighted = tf.nn.sigmoid_cross_entropy_with_logits(labels=y_true, logits=y_pred_logits)
#     l_w_bce = tf.where(tf.equal(y_true, 1), bce_unweighted * self.w1, bce_unweighted * self.w0)

#     l_amp = tf.zeros_like(y_pred_logits)
#     penalty_minority = self.lambda1 * tf.pow(tf.maximum(0., self.m1 - y_pred_logits), self.beta1)
#     l_amp = tf.where(tf.equal(y_true, 1), penalty_minority, l_amp)

#     penalty_majority = self.lambda0 * tf.pow(tf.maximum(0., y_pred_logits - self.m0), self.beta0)
#     l_amp = tf.where(tf.equal(y_true, 0), penalty_majority, l_amp)

#     total_loss = tf.reduce_mean(l_w_bce + l_amp)

#     return total_loss

#   @property
#   def trainable_variables(self):
#       # Expose trainable variables for optimizer
#     return [self.m1, self.m0, self.lambda1, self.lambda0]

# class FocalLossTF(tf.keras.losses.Loss):
#   """
#   Focal Loss for Imbalanced Datasets in TensorFlow.
#   Focuses training on hard, misclassified examples by down-weighting easy examples.
#   """
#   def __init__(self, alpha=0.25, gamma=2.0, name="focal_loss_tf"):
#     super().__init__(name=name)
#     self.alpha = tf.constant(alpha, dtype=tf.float32)
#     self.gamma = tf.constant(gamma, dtype=tf.float32)

#   def call(self, y_true, y_pred_logits):
#     y_true = tf.cast(y_true, tf.float32)
#     y_pred_logits = tf.cast(y_pred_logits, tf.float32)

#     # Calculate standard BCE (log loss)
#     bce = tf.nn.sigmoid_cross_entropy_with_logits(labels=y_true, logits=y_pred_logits)

#     # Calculate probabilities
#     p = tf.sigmoid(y_pred_logits)

#     # p_t is the probability of the true class
#     p_t = y_true * p + (1 - y_true) * (1 - p)

#     # alpha_t is alpha for positive, 1-alpha for negative
#     alpha_t = y_true * self.alpha + (1 - y_true) * (1.0 - self.alpha)

#     # Modulating factor (1 - p_t)^gamma
#     modulating_factor = tf.pow(1.0 - p_t, self.gamma)

#     # Focal Loss: alpha_t * (1 - p_t)^gamma * BCE
#     focal_loss = alpha_t * modulating_factor * bce
#     return tf.reduce_mean(focal_loss)

# class ClassBalancedLossTF(tf.keras.losses.Loss):
#   """
#   Class Balanced Loss for Imbalanced Datasets in TensorFlow.
#   Re-weights the standard loss by the 'effective number of samples' for each class.
#   """
#   def __init__(self, num_majority, num_minority, beta=0.999, name="class_balanced_loss_tf"):
#     super().__init__(name=name)
#     self.beta = tf.constant(beta, dtype=tf.float32)

#     # Calculate effective number of samples for each class
#     # Add a small epsilon to avoid division by zero in case num_samples is 0 or beta is 1
#     epsilon = tf.keras.backend.epsilon()

#     effective_num_majority = (1.0 - tf.pow(self.beta, tf.cast(num_majority, tf.float32))) / (1.0 - self.beta + epsilon)
#     effective_num_minority = (1.0 - tf.pow(self.beta, tf.cast(num_minority, tf.float32))) / (1.0 - self.beta + epsilon)

#     # Calculate class-balanced weights
#     self.w0_cb = tf.constant(1.0, dtype=tf.float32) / effective_num_majority
#     self.w1_cb = tf.constant(1.0, dtype=tf.float32) / effective_num_minority

#     tf.print(f"CB Loss Weights: w0 (majority)={self.w0_cb}, w1 (minority)={self.w1_cb}")

#   def call(self, y_true, y_pred_logits):
#     y_true = tf.cast(y_true, tf.float32)
#     y_pred_logits = tf.cast(y_pred_logits, tf.float32)

#     # Calculate standard BCE
#     bce_unweighted = tf.nn.sigmoid_cross_entropy_with_logits(labels=y_true, logits=y_pred_logits)

#     # Apply class-balanced weights
#     cb_loss = tf.where(tf.equal(y_true, 1), bce_unweighted * self.w1_cb, bce_unweighted * self.w0_cb)

#     return tf.reduce_mean(cb_loss)


# # --- 6. Main Execution Block ---
# if __name__ == "__main__":
#     # List of loss functions to compare
#     LOSS_TYPES_TO_COMPARE = ['AMRL', 'BCE', 'FOCAL', 'CB']


#     all_test_results = [] # To store results from all loss functions - This will no longer be used for saving all results

#     # --- Configuration for data loading ---
#     # This script is now configured to ONLY load from Google Drive.
#     # Please uncomment and configure GOOGLE_DRIVE_PATH

#     # Ensure GOOGLE_DRIVE_PATH is set and exists
#     if 'GOOGLE_DRIVE_PATH' not in globals() or not os.path.exists(GOOGLE_DRIVE_PATH):
#         raise ValueError("GOOGLE_DRIVE_PATH not set or does not exist. Please configure it to your Google Drive folder containing CSV files.")

#     print(f"Scanning for XLSX files in: {GOOGLE_DRIVE_PATH}")


#     file_paths_to_process = [os.path.join(GOOGLE_DRIVE_PATH, f) for f in os.listdir(GOOGLE_DRIVE_PATH) if f.endswith('.xlsx')]
#     # sort the files
#     file_paths_to_process = sorted(file_paths_to_process, key=lambda x: os.path.basename(x).lower())

#     if not file_paths_to_process:
#         print(f"No XLSX files found in {GOOGLE_DRIVE_PATH}. Please check the path and file extensions.")
#         exit()

#     for file_path in file_paths_to_process:
#         current_file_name = os.path.basename(file_path)
#         print(f"\n############################################################")
#         print(f"Processing Dataset: {current_file_name}")
#         print(f"############################################################")

#         # Load data for the current file
#         df = load_data_from_drive(file_path)
#         if df is None:
#             print(f"Skipping {current_file_name} due to loading error.")
#             continue # Skip to the next file

#         # Define features (X) and target (y)
#         # Assumes the last column is always the target
#         X = df.iloc[:, :-1]
#         y = df.iloc[:, -1]

#         # --- Data Preprocessing and Splitting (for current dataset) ---
#         print("\nSplitting data into train, validation, and test sets...")
#         X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42, stratify = y)
#         X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = (0.1/0.8), random_state=42, stratify = y_train)

#         print(f"Train set size: {len(X_train)} (Minority: {y_train.sum()})")
#         print(f"Validation set size: {len(X_val)} (Minority: {y_val.sum()})")
#         print(f"Test set size: {len(X_test)} (Minority: {y_test.sum()})")

#         scaler = StandardScaler()
#         X_train_scaled = scaler.fit_transform(X_train)
#         X_val_scaled = scaler.transform(X_val)
#         X_test_scaled = scaler.transform(X_test)

#         y_train_tf = y_train.values.astype(np.float32).reshape(-1, 1)
#         y_val_tf = y_val.values.astype(np.float32).reshape(-1, 1)
#         y_test_tf = y_test.values.astype(np.float32).reshape(-1, 1)


#         train_dataset_tf = tf.data.Dataset.from_tensor_slices((X_train_scaled.astype(np.float32), y_train_tf)).batch(BATCH_SIZE).shuffle(buffer_size=len(X_train_scaled))
#         val_dataset_tf = tf.data.Dataset.from_tensor_slices((X_val_scaled.astype(np.float32), y_val_tf)).batch(BATCH_SIZE)
#         test_dataset_tf = tf.data.Dataset.from_tensor_slices((X_test_scaled.astype(np.float32), y_test_tf)).batch(BATCH_SIZE)

#         # --- Loss Function Parameters (common calculations for current dataset) ---
#         num_minority = np.sum(y_train)
#         num_majority = len(y_train) - num_minority


#         w1_bce_amrl = num_majority / (num_minority + num_majority)
#         w0_bce_amrl = num_minority / (num_minority + num_majority)

#         # Parameters for Focal Loss
#         alpha_focal = num_majority / (num_minority + num_majority)
#         gamma_focal = 2.0

#         # Parameters for Class Balanced Loss
#         beta_cb = 0.999

#         # Calculate input_dim from the scaled training data shape
#         input_dim = X_train_scaled.shape[1]
#         print(f"Input dimension for the model: {input_dim}")

#         loss_results_for_file = [] # List to store results for each loss function for the current file

#         for loss_type in LOSS_TYPES_TO_COMPARE:
#             print(f"\n--------------------------------------------------------")
#             print(f"Training {current_file_name} with {loss_type} Loss")
#             print(f"--------------------------------------------------------")

#             # Re-initialize the model for each loss function to ensure a fresh start
#             model = SimpleMLP_TF(input_dim)

#             loss_function = None


#             if loss_type == 'AMRL':
#                 print("Margin Loss Parameters:")
#                 print(f"  BCE Weights: w1_bce (minority) = {w1_bce_amrl:.4f}, w0_bce (majority) = {w0_bce_amrl:.4f}")
#                 print(f"  Margins: m1 (pos) = {M_POS}, m0 (neg) = {M_NEG}")
#                 print(f"  Lambdas: lambda1 (pos) = {LAMBDA_POS}, lambda0 (neg) = {LAMBDA_NEG}")
#                 print(f"  Betas: beta1 (pos) = {BETA_POS}, beta0 (neg) = {BETA_NEG}")
#                 loss_function = AMRLLossTF(w1 = w1_bce_amrl, w0 = w0_bce_amrl, m1 = M_POS, m0 = M_NEG,
#                                           lambda1=LAMBDA_POS, lambda0 = LAMBDA_NEG,
#                                           beta1 = BETA_POS, beta0 = BETA_NEG)
#             elif loss_type == 'BCE':
#                 print("Standard Binary Crossentropy Loss")
#                 loss_function = tf.keras.losses.BinaryCrossentropy(from_logits = True)
#             elif loss_type == 'FOCAL':
#                 print(f"Focal Loss Parameters: alpha={alpha_focal:.4f}, gamma={gamma_focal:.4f}")
#                 loss_function = FocalLossTF(alpha = alpha_focal, gamma = gamma_focal)
#             elif loss_type == 'CB':
#                 print(f"Class Balanced Loss Parameters: beta={beta_cb:.4f}")
#                 loss_function = ClassBalancedLossTF(num_majority = num_majority, num_minority = num_minority, beta = beta_cb)
#             else:
#                 raise ValueError(f"Invalid LOSS_TYPE: {loss_type}. Choose from 'AMRL', 'BCE', 'FOCAL', 'CB'.")


#             # Compile the model with the selected loss and relevant metrics
#             model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
#                           loss=loss_function,
#                           metrics=[
#                               BinaryAccuracy(name = 'accuracy', threshold = 0.5),
#                               Precision(name = 'precision', thresholds = 0.5),
#                               Recall(name = 'recall', thresholds = 0.5),
#                               AUC(name = 'auc_roc'), # ROC AUC
#                               AUC(curve = 'PR', name =' auc_pr') # Precision-Recall AUC
#                           ])


#             reduce_lr = ReduceLROnPlateau(
#                                           monitor = 'val_loss',     # metric to monitor
#                                           factor = 0.5,             # reduce LR by this factor
#                                           patience = 3,             # wait 3 epochs with no improvement
#                                           min_lr = 1e-6,            # lower bound on LR
#                                           verbose = 1
#                                           )

#             print("\nStarting training...")
#             history = model.fit(train_dataset_tf,
#                                 epochs = NUM_EPOCHS,
#                                 validation_data = val_dataset_tf,
#                                 verbose = 0,
#                                 callbacks = [reduce_lr]) # Set verbose to 0 to reduce output during batch runs

#             print("Training finished.")

#             print("\nEvaluating on the Test Set...")
#             # Get raw logits and true labels for full metric calculation
#             y_pred_logits_test = model.predict(X_test_scaled.astype(np.float32))
#             y_pred_probs_test = tf.sigmoid(y_pred_logits_test).numpy().flatten()
#             y_pred_binary_test = (y_pred_probs_test > 0.5).astype(int)
#             y_true_test_flat = y_test.values.flatten()

#             # Calculate additional metrics
#             # Confusion Matrix components
#             tn, fp, fn, tp = confusion_matrix(y_true_test_flat, y_pred_binary_test, labels=[0, 1]).ravel()

#             # Specificity (True Negative Rate)
#             specificity = tn / (tn + fp) if (tn + fp) > 0 else 0.0

#             # Geometric Mean (G-mean)
#             # Recall is already available from Keras metrics
#             #current_recall = model.evaluate(test_dataset_tf, verbose=0)[3] # Index 3 is 'recall'
#             current_recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
#             g_mean = np.sqrt(current_recall * specificity) if current_recall >= 0 and specificity >= 0 else np.nan

#             # Balanced Accuracy
#             bal_accuracy = balanced_accuracy_score(y_true_test_flat, y_pred_binary_test)

#             # Kuipers Skill Score (KSS) / True Skill Score (TSS)
#             kss_tss = current_recall + specificity - 1

#             # Heidke Skill Score (HSS)
#             # Expected Accuracy (chance accuracy)
#             total_samples = len(y_true_test_flat)
#             p_pos_true = np.sum(y_true_test_flat == 1) / total_samples
#             p_neg_true = np.sum(y_true_test_flat == 0) / total_samples
#             p_pos_pred = np.sum(y_pred_binary_test == 1) / total_samples
#             p_neg_pred = np.sum(y_pred_binary_test == 0) / total_samples

#             expected_accuracy = (p_pos_true * p_pos_pred) + (p_neg_true * p_neg_pred)

#             current_accuracy = accuracy_score(y_true_test_flat, y_pred_binary_test) # Recalculate accuracy for consistency
#             hss = (current_accuracy - expected_accuracy) / (1 - expected_accuracy) if (1 - expected_accuracy) != 0 else np.nan


#             print(f"\n--- Test Set Results for {loss_type} on {current_file_name} ---")
#             # Get standard Keras metrics from the last evaluation
#             keras_metrics = model.evaluate(test_dataset_tf, verbose=0)
#             metric_names = ['Loss', 'Accuracy', 'Precision', 'Recall', 'AUC-ROC', 'AUC-PR']
#             test_metrics_dict = dict(zip(metric_names, keras_metrics))

#             # Add new metrics
#             test_metrics_dict['Specificity'] = specificity
#             test_metrics_dict['G-Mean'] = g_mean
#             test_metrics_dict['Balanced_Accuracy'] = bal_accuracy
#             test_metrics_dict['KSS_TSS'] = kss_tss
#             test_metrics_dict['HSS'] = hss

#             # Add dataset name and loss type
#             test_metrics_dict['Dataset'] = current_file_name
#             test_metrics_dict['Loss_Type'] = loss_type
#             #all_test_results.append(test_metrics_dict) # No longer appending to the overall list

#             for metric, value in test_metrics_dict.items():
#                 if isinstance(value, float):
#                     print(f"{metric}: {value:.4f}")
#                 else:
#                     print(f"{metric}: {value}")

#             # --- Consolidate and Save All Results ---
#             #results_df = pd.DataFrame(all_test_results) # This will now be done per file

#             # Define the path to save the results file
#             base_path = globals().get('GOOGLE_DRIVE_PATH', './') # Defaults to current directory if GOOGLE_DRIVE_PATH not set

#             # Save this loss result in the list
#             loss_results_for_file.append(test_metrics_dict.copy())

#         # --- Save all loss results for the current file ---
#         file_results_df = pd.DataFrame(loss_results_for_file)
#         output_filename = f"{current_file_name}_all_losses.csv"
#         output_filepath = os.path.join(base_path, output_filename)
#         file_results_df.to_csv(output_filepath, index=False)

#         print(f"\nResults for {current_file_name} saved to: {output_filepath}")
#         print("\nComprehensive Comparison Summary for this file:")
#         print(file_results_df.round(4).to_string())

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.metrics import BinaryAccuracy, Precision, Recall, AUC
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, accuracy_score
import os
from tensorflow.keras.callbacks import EarlyStopping # Import EarlyStopping callback
from tensorflow.keras.callbacks import ReduceLROnPlateau # Keep import in case it's used elsewhere, but not needed for custom loop LR reduction
from google.colab import drive # Import drive here

from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, average_precision_score

import matplotlib.pyplot as plt # Import matplotlib for plotting


from tensorflow.keras.models import save_model

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Define your Google Drive base path (adjust as needed)
GOOGLE_DRIVE_PATH = '/content/drive/MyDrive/1. Trial/'

# Training parameters
BATCH_SIZE = 5
NUM_EPOCHS = 10

# Parameters for AMRL (initial values – these will be trained)
M_POS_INIT = 1.0
M_NEG_INIT = -1.0
LAMBDA_POS_INIT = 2.0
LAMBDA_NEG_INIT = 1.0
BETA_POS_INIT = 2
BETA_NEG_INIT = 1

# Early stopping parameters
EARLY_STOPPING_PATIENCE = 20  # Number of epochs with no improvement before stopping
MIN_DELTA = 0.0001            # Minimum improvement to be considered as progress

# Gradient clipping parameter
CLIP_NORM = 1.0  # Maximum L2 norm of the gradients

# GAN training stability parameters
ADD_NOISE = True           # Add noise to discriminator inputs
NOISE_STDDEV = 0.1         # Standard deviation of the noise
DISC_TRAIN_RATIO = 1       # Number of discriminator steps per generator step
GAN_LAMBDA = 0.1           # Weighting factor for adversarial loss in generator



In [ ]:
"""#FALSE_ALARM_RATE(FAR)=Ratio of the number of false prediction of danger and the total number of the model prediction when actually no danger was observed.
                                  #FAR=b/(b+d);0=<FAR<=1"""
def false_alarm(tp,tn,fp,fn):

    false_alarm = fp / (fp + tn)
    false_alarm = round(false_alarm, 2)

#print('The false alarm value of model is:',false_alarm)
    return false_alarm




"""#BIAS_SCORE=Ratio of the number of days whe the model predicted 'danger and the total number of avalanche occurences events.
                                  #B=(a+b)/(a+c)
                                  #B=1 -> Unbiased
                                  #B<=1 -> Underforcast
                                  #B>=1 -> Overforecast"""
def bias_score(tp,tn,fp,fn):

    bias_score = (tp + fp) / (tp + fn)
    bias_score = round(bias_score, 2)
    #print('The bias score of model is:',bias_score)
    return bias_score




"""#Probability of detection(PoD)=Probability that the event was forecast when it occurred.
                                  #PoD=a/(a+c);0=<Pod<=1"""
def probability_of_detection(tp,tn,fp,fn):
    pod_value = tp / (tp + fn)
    pod_value = round(pod_value, 2)
    #print('The POD value of model is:',pod_value)
    return pod_value




"""#Heidke_skill_score(HSS)=Skill score based on hit rate:HSS=(Hit + Correct Negative - Chance)/(Total - Chance), where chance is the expected number of correct event forecast due to chance.
                            #HSS=2(ad-bc)/((a+b)(b+d)+(a+c)(c+d)); (-)infinity=<HSS<=1 """
def heidke_skill_score(tp,tn,fp,fn):
    heidke_skill_value = 2 * ((tp * tn) - (fn * fp)) /(((tp + fn) * (fn + tn)) + ((tp + fp) * (fp + tn)))
    heidke_skill_value = round(heidke_skill_value,2)
    #print('The HSS score of model is:',heidke_skill_value)
    return heidke_skill_value


"""#Pierce_skill_score(PSS)=Skill score based on hit rate:HSS=(Hit + Correct Negative - Chance)/(Total - Chance), where chance is the expected number of correct event forecast due to chance.
                            #HSS=2(ad-bc)/((a+b)(b+d)+(a+c)(c+d)); (-)infinity=<HSS<=1 """
def pierce_skill_score(tp,tn,fp,fn):
    pierce_skill_value = (tp / (tp + fn)) - (fp / (tp + tn))
    pierce_skill_value = round(pierce_skill_value,2)
    #print('The PSS score of model is:',pierce_skill_value)
    return pierce_skill_value

def f1_score(tp,tn,fp,fn):
    f1_value = (2*(tp/(tp+fp))*(tp/(tp+fn)))/((tp/(tp+fp)) + (tp/(tp+fn)))
    f1_value = round(f1_value, 2)
    #print('The F1-Score of the model is:',f1_value)
    return f1_value

In [ ]:
# Function to compute class ratios
def get_class_ratio(y, decimals=2):
    counts = np.bincount(y)
    total = len(y)
    # Ensure counts has at least two elements for binary classification
    if len(counts) < 2:
        if 0 in y:
            counts = np.array([counts[0], 0])
        else:
            counts = np.array([0, counts[0]])

    ratio = {cls: round(counts[cls] / total, decimals) for cls in range(len(counts))}

    # Compute imbalance ratio (majority/minority)
    majority_count = max(counts)
    minority_count = min(counts)
    if minority_count > 0:
        imbalance_ratio = f"{round(majority_count / minority_count, decimals)} : 1"
    elif majority_count > 0:
         imbalance_ratio = "inf : 1" # All samples are majority
    else:
         imbalance_ratio = "0 : 0" # No samples


    return counts, ratio, imbalance_ratio

In [ ]:
# Function to plot training history
def plot_training_history(history, dataset_name, loss_type, save_dir):
    """
    Plots training and validation metrics from the history object.
    Saves the plots to a specified directory.
    """
    # Handle cases where history might be empty or None
    if not history:
        print(f"Warning: No history data available to plot for {loss_type} on {dataset_name}.")
        return

    # Get epochs from one of the history lists (assuming all have the same length)
    # Check if any list in history is non-empty before getting length
    first_key = next(iter(history), None)
    if not first_key or not history[first_key]:
         print(f"Warning: History dictionary is empty or its lists are empty for {loss_type} on {dataset_name}. Cannot plot.")
         return

    epochs = range(1, len(history[first_key]) + 1)

    # Define metrics to plot based on availability in history keys
    metrics_to_plot = [key for key in history if not key.startswith('val_') and key != 'epoch']
    val_metrics_to_plot = [key for key in history if key.startswith('val_')]


    for metric_name in metrics_to_plot:
        val_metric_name = f'val_{metric_name}'
        if val_metric_name in history:
            plt.figure(figsize=(10, 6))
            plt.plot(epochs, history[metric_name], 'b', label=f'Training {metric_name.replace("_", " ")}')
            plt.plot(epochs, history[val_metric_name], 'r', label=f'Validation {val_metric_name.replace("val_", "").replace("_", " ")}')
            plt.title(f'{metric_name.replace("_", " ").capitalize()} for {loss_type} on {dataset_name}')
            plt.xlabel('Epochs')
            plt.ylabel(metric_name.replace("_", " ").capitalize())
            plt.legend()
            plt.grid(True)

            # Sanitize filename
            safe_dataset_name = dataset_name.replace('.xlsx', '').replace(' ', '_').replace('-', '_')
            safe_loss_type = loss_type.replace('-', '_')
            filename = f"{safe_dataset_name}_{safe_loss_type}_{metric_name}_plot.png"
            save_path = os.path.join(save_dir, filename)

            try:
                plt.savefig(save_path)
                print(f"Saved plot to: {save_path}")
            except Exception as e:
                print(f"Error saving plot {filename}: {e}")

            plt.close() # Close the figure to free up memory

In [ ]:
# Custom train step for AMRL with gradient clipping and lambda clipping
# Removed @tf.function decorator
def train_step_amrl(model, loss_function, optimizer, X_batch, y_batch, clip_norm):
    with tf.GradientTape() as tape:
        y_pred_logits = model(X_batch, training=True) # Set training=True for dropout
        loss = loss_function(y_batch, y_pred_logits)

    # Get gradients for both model's trainable variables AND loss function's trainable variables
    # Ensure all trainable variables are collected
    trainable_variables = model.trainable_variables + loss_function.trainable_variables if hasattr(loss_function, 'trainable_variables') else model.trainable_variables
    gradients = tape.gradient(loss, trainable_variables)

    # Check for None gradients and filter
    if any(grad is None for grad in gradients):
        none_grads = [var.name for grad, var in zip(gradients, trainable_variables) if grad is None]
        # print(f"Warning: Gradients are None for variables: {none_grads}") # Keep this for debugging if needed
        grads_and_vars = [(grad, var) for grad, var in zip(gradients, trainable_variables) if grad is not None]
        if not grads_and_vars:
             # This should ideally not happen if loss and model are connected
            raise ValueError("All gradients are None. Cannot apply gradients.")
    else:
        grads_and_vars = list(zip(gradients, trainable_variables))


    # Apply gradient clipping
    clipped_gradients, _ = tf.clip_by_global_norm([g for g, v in grads_and_vars], clip_norm)

    # Apply gradients
    optimizer.apply_gradients(zip(clipped_gradients, [v for g, v in grads_and_vars]))

    # Manually clip lambda values to be non-negative for AMRLLossTF after applying gradients
    if isinstance(loss_function, AMRLLossTF):
         if hasattr(loss_function, 'lambda1') and hasattr(loss_function, 'lambda0'):
            loss_function.lambda1.assign(tf.maximum(0., loss_function.lambda1.numpy()))
            loss_function.lambda0.assign(tf.maximum(0., loss_function.lambda0.numpy()))


    return loss

# Custom test step for AMRL (used for validation)
# Removed @tf.function decorator
def test_step_amrl(model, loss_function, X_batch, y_batch, metrics):
    y_pred_logits = model(X_batch, training=False) # Set training=False for dropout
    loss = loss_function(y_batch, y_pred_logits)

    # Update metrics
    y_pred_probs = tf.sigmoid(y_pred_logits)
    metrics['accuracy'].update_state(y_batch, y_pred_probs)
    metrics['precision'].update_state(y_batch, y_pred_probs)
    metrics['recall'].update_state(y_batch, y_pred_probs)
    metrics['auc_roc'].update_state(y_batch, y_pred_probs)
    metrics['auc_pr'].update_state(y_batch, y_pred_probs)

    return loss


# Custom training loop for AMRL
def custom_train_loop(model, loss_function, optimizer, train_dataset, val_dataset, epochs, early_stopping, clip_norm):
    best_val_loss = float('inf')
    epochs_without_improvement = 0
    # Define paths for saving temporary weights
    model_weights_path = 'temp_model_weights.weights.h5' # Corrected filename extension
    loss_weights_path = 'temp_loss_weights.npy'


    history = {
        'loss': [], 'accuracy': [], 'precision': [], 'recall': [], 'auc_roc': [], 'auc_pr': [],
        'val_loss': [], 'val_accuracy': [], 'val_precision': [], 'val_recall': [], 'val_auc_roc': [], 'val_auc_pr': []
    }

    # Initialize metrics for tracking within the custom loop
    train_metrics = {
        'accuracy': BinaryAccuracy(threshold=0.5),
        'precision': Precision(thresholds=0.5),
        'recall': Recall(thresholds=0.5),
        'auc_roc': AUC(),
        'auc_pr': AUC(curve='PR')
    }
    val_metrics = {
        'accuracy': BinaryAccuracy(threshold=0.5),
        'precision': Precision(thresholds=0.5),
        'recall': Recall(thresholds=0.5),
        'auc_roc': AUC(),
        'auc_pr': AUC(curve='PR')
    }


    for epoch in range(epochs):
        # print(f"\nEpoch {epoch+1}/{epochs}") # Reduced verbosity during batch processing

        # Reset metrics at the start of each epoch
        for metric in train_metrics.values():
            metric.reset_state()
        for metric in val_metrics.values():
            metric.reset_state()

        total_train_loss = 0
        num_train_batches = 0
        for X_batch, y_batch in train_dataset:
            loss = train_step_amrl(model, loss_function, optimizer, X_batch, y_batch, clip_norm)
            total_train_loss += loss.numpy()
            num_train_batches += 1

            # Update train metrics (need to manually update here as train_step doesn't return probabilities/predictions)
            # Use training=False to get deterministic predictions for metrics evaluation on the training data batch
            y_pred_logits = model(X_batch, training=False)
            y_pred_probs = tf.sigmoid(y_pred_logits)
            train_metrics['accuracy'].update_state(y_batch, y_pred_probs)
            train_metrics['precision'].update_state(y_batch, y_pred_probs)
            train_metrics['recall'].update_state(y_batch, y_pred_probs)
            train_metrics['auc_roc'].update_state(y_batch, y_pred_probs)
            train_metrics['auc_pr'].update_state(y_batch, y_pred_probs)


        avg_train_loss = total_train_loss / num_train_batches

        total_val_loss = 0
        num_val_batches = 0
        for X_batch_val, y_batch_val in val_dataset:
            loss = test_step_amrl(model, loss_function, X_batch_val, y_batch_val, val_metrics)
            total_val_loss += loss.numpy()
            num_val_batches += 1

        avg_val_loss = total_val_loss / num_val_batches

        # Get metric results
        train_acc = train_metrics['accuracy'].result().numpy()
        train_prec = train_metrics['precision'].result().numpy()
        train_recall = train_metrics['recall'].result().numpy()
        train_auc_roc = train_metrics['auc_roc'].result().numpy()
        train_auc_pr = train_metrics['auc_pr'].result().numpy()

        val_acc = val_metrics['accuracy'].result().numpy()
        val_prec = val_metrics['precision'].result().numpy()
        val_recall = val_metrics['recall'].result().numpy()
        val_auc_roc = val_metrics['auc_roc'].result().numpy()
        val_auc_pr = val_metrics['auc_pr'].result().numpy()

        # Append metrics to history for plotting
        history['loss'].append(avg_train_loss)
        history['accuracy'].append(train_acc)
        history['precision'].append(train_prec)
        history['recall'].append(train_recall)
        history['auc_roc'].append(train_auc_roc)
        history['auc_pr'].append(train_auc_pr)
        history['val_loss'].append(avg_val_loss)
        history['val_accuracy'].append(val_acc)
        history['val_precision'].append(val_prec)
        history['val_recall'].append(val_recall)
        history['val_auc_roc'].append(val_auc_roc)
        history['val_auc_pr'].append(val_auc_pr)


        # Print epoch summary (reduced verbosity)
        # print(f"  Train Loss: {avg_train_loss:.4f}, Train Acc: {train_acc:.4f}, Train Prec: {train_prec:.4f}, Train Recall: {train_recall:.4f}, Train AUC-ROC: {train_auc_roc:.4f}, Train AUC-PR: {train_auc_pr:.4f}")
        # print(f"  Val Loss: {avg_val_loss:.4f}, Val Acc: {val_acc:.4f}, Val Prec: {val_prec:.4f}, Val Recall: {val_recall:.4f}, Val AUC-ROC: {val_auc_roc:.4f}, Val AUC-PR: {val_auc_pr:.4f}")


        # Print the current values of the trainable loss parameters (only for AMRL)
        if isinstance(loss_function, AMRLLossTF):
            learned_params = [loss_function.m1.numpy(), loss_function.m0.numpy(), loss_function.lambda1.numpy(), loss_function.lambda0.numpy()]
            # print(f"  AMRL Learned Params: m1={learned_params[0]:.4f}, m0={learned_params[1]:.4f}, lambda1={learned_params[2]:.4f}, lambda0={learned_params[3]:.4f}") # Reduced verbosity


        # --- Early Stopping Check (using validation loss) ---
        # Check if validation loss improved by at least min_delta
        if avg_val_loss < best_val_loss - early_stopping.min_delta:
            best_val_loss = avg_val_loss
            epochs_without_improvement = 0
            # Save current weights if validation loss improved
            model.save_weights(model_weights_path)
            # Save loss function trainable variables (only for AMRL)
            if isinstance(loss_function, AMRLLossTF):
                 np.save(loss_weights_path, learned_params)
            # print("Saved model and loss weights.") # Reduced verbosity

        else:
            epochs_without_improvement += 1
            if epochs_without_improvement >= early_stopping.patience:
                print(f"Early stopping triggered based on validation loss after {epoch+1} epochs.")
                # Restore best weights if early stopping is triggered
                if os.path.exists(model_weights_path):
                    print("Restoring model weights from best validation epoch...")
                    model.load_weights(model_weights_path)
                # Restore loss parameters (only for AMRL)
                if isinstance(loss_function, AMRLLossTF) and os.path.exists(loss_weights_path):
                     print("Restoring loss parameters from best validation epoch...")
                     best_loss_params = np.load(loss_weights_path)
                     loss_function.m1.assign(best_loss_params[0])
                     loss_function.m0.assign(best_loss_params[1])
                     loss_function.lambda1.assign(best_loss_params[2])
                     loss_function.lambda0.assign(best_loss_params[3])
                     # print(f"  Restored AMRL Params: m1={loss_function.m1.numpy():.4f}, m0={loss_function.m0.numpy():.4f}, lambda1={loss_function.lambda1.numpy():.4f}, lambda0={loss_function.lambda0.numpy():.4f}") # Reduced verbosity

                break # Exit the training loop

    return history # Return the history dictionary

In [ ]:
class AMRLLossTF(tf.keras.losses.Loss):
    """
    Margin Loss for Imbalanced Datasets in TensorFlow.
    Combines Weighted Binary Cross-Entropy with an Asymmetric Margin Penalty.
    """
    def __init__(self, w1, w0, m1_init, m0_init, lambda1_init, lambda0_init, beta1=1, beta0=1, name="margin_loss_tf"):
        super().__init__(name=name)
        self.w1 = tf.Variable(w1, dtype=tf.float32, trainable=False) # BCE weights are not trainable
        self.w0 = tf.Variable(w0, dtype=tf.float32, trainable=False) # BCE weights are not trainable
        # Make margins and lambdas trainable variables as per original design intent
        self.m1 = tf.Variable(m1_init, dtype=tf.float32, trainable=True, name="m1")
        self.m0 = tf.Variable(m0_init, dtype=tf.float32, trainable=True, name="m0")
        self.lambda1 = tf.Variable(lambda1_init, dtype=tf.float32, trainable=True, name="lambda1")
        self.lambda0 = tf.Variable(lambda0_init, dtype=tf.float32, trainable=True, name="lambda0")
        self.beta1 = tf.constant(float(beta1), dtype=tf.float32) # Betas are constants
        self.beta0 = tf.constant(float(beta0), dtype=tf.float32) # Betas are constants

    def call(self, y_true, y_pred_logits):
        y_true = tf.cast(y_true, tf.float32)
        y_pred_logits = tf.cast(y_pred_logits, tf.float32)

        bce_unweighted = tf.nn.sigmoid_cross_entropy_with_logits(labels=y_true, logits=y_pred_logits)
        l_w_bce = tf.where(tf.equal(y_true, 1), bce_unweighted * self.w1, bce_unweighted * self.w0)

        l_amp = tf.zeros_like(y_pred_logits)
        penalty_minority = self.lambda1 * tf.pow(tf.maximum(0., self.m1 - y_pred_logits), self.beta1)
        l_amp = tf.where(tf.equal(y_true, 1), penalty_minority, l_amp)

        penalty_majority = self.lambda0 * tf.pow(tf.maximum(0., y_pred_logits - self.m0), self.beta0)
        l_amp = tf.where(tf.equal(y_true, 0), penalty_majority, l_amp)

        total_loss = tf.reduce_mean(l_w_bce + l_amp)

        # Compare individual components:

        #print("Weighted BCE:", tf.reduce_mean(l_w_bce).numpy(), "AMP:", tf.reduce_mean(l_amp).numpy())

        #print("Margins and Lambdas:", self.m1.numpy(), self.m0.numpy(), self.lambda1.numpy(), self.lambda0.numpy())



        return total_loss

    @property
    def trainable_variables(self):
        # Explicitly list the trainable variables of the loss function
        return [self.m1, self.m0, self.lambda1, self.lambda0]

# --- Custom Training Step for AMRL Loss ---
# Removed @tf.function
def train_step_amrl(model, loss_function, optimizer, X_batch, y_batch, clip_norm):
    with tf.GradientTape() as tape:
        y_pred_logits = model(X_batch, training=True) # Set training=True for dropout
        loss = loss_function(y_batch, y_pred_logits)

    # Get gradients for both model's trainable variables AND loss function's trainable variables
    trainable_variables = model.trainable_variables + loss_function.trainable_variables
    gradients = tape.gradient(loss, trainable_variables)

    # Apply gradient clipping
    clipped_gradients, _ = tf.clip_by_global_norm(gradients, clip_norm)

    optimizer.apply_gradients(zip(clipped_gradients, trainable_variables))

    return loss

# --- Custom Test Step for AMRL Loss Evaluation ---
# Removed @tf.function
def test_step_amrl(model, loss_function, X_batch, y_batch, metrics):
    y_pred_logits = model(X_batch, training=False) # Set training=False for dropout
    loss = loss_function(y_batch, y_pred_logits)

    # Update metrics
    y_pred_probs = tf.sigmoid(y_pred_logits)
    metrics['accuracy'].update_state(y_batch, y_pred_probs)
    metrics['precision'].update_state(y_batch, y_pred_probs)
    metrics['recall'].update_state(y_batch, y_pred_probs)
    metrics['auc_roc'].update_state(y_batch, y_pred_probs)
    metrics['auc_pr'].update_state(y_batch, y_pred_probs)

    return loss


# --- Custom Training Loop for AMRL Loss ---
def custom_train_loop(model, loss_function, optimizer, train_dataset, val_dataset, epochs, early_stopping, clip_norm):
    best_val_loss = float('inf')
    epochs_without_improvement = 0
    # Define paths for saving temporary weights
    model_weights_path = 'temp_model_weights.weights.h5' # Corrected filename extension
    loss_weights_path = 'temp_loss_weights.npy'

    # Lists to store metric history for plotting
    history = {
        'loss': [], 'accuracy': [], 'precision': [], 'recall': [], 'auc_roc': [], 'auc_pr': [],
        'val_loss': [], 'val_accuracy': [], 'val_precision': [], 'val_recall': [], 'val_auc_roc': [], 'val_auc_pr': []
    }

    # Initialize metrics for tracking within the custom loop
    train_metrics = {
        'accuracy': BinaryAccuracy(threshold=0.5),
        'precision': Precision(thresholds=0.5),
        'recall': Recall(thresholds=0.5),
        'auc_roc': AUC(),
        'auc_pr': AUC(curve='PR')
    }
    val_metrics = {
        'accuracy': BinaryAccuracy(threshold=0.5),
        'precision': Precision(thresholds=0.5),
        'recall': Recall(thresholds=0.5),
        'auc_roc': AUC(),
        'auc_pr': AUC(curve='PR')
    }


    for epoch in range(epochs):
        print(f"\nEpoch {epoch+1}/{epochs}")

        # Reset metrics at the start of each epoch
        for metric in train_metrics.values():
            metric.reset_state()
        for metric in val_metrics.values():
            metric.reset_state()

        total_train_loss = 0
        num_train_batches = 0
        for X_batch, y_batch in train_dataset:
            loss = train_step_amrl(model, loss_function, optimizer, X_batch, y_batch, clip_norm)
            total_train_loss += loss.numpy()
            num_train_batches += 1

            # Update train metrics (need to manually update here as train_step doesn't return probabilities/predictions)
            y_pred_logits = model(X_batch, training=False) # Use training=False to get deterministic predictions for metrics
            y_pred_probs = tf.sigmoid(y_pred_logits)
            train_metrics['accuracy'].update_state(y_batch, y_pred_probs)
            train_metrics['precision'].update_state(y_batch, y_pred_probs)
            train_metrics['recall'].update_state(y_batch, y_pred_probs)
            train_metrics['auc_roc'].update_state(y_batch, y_pred_probs)
            train_metrics['auc_pr'].update_state(y_batch, y_pred_probs)


        avg_train_loss = total_train_loss / num_train_batches

        total_val_loss = 0
        num_val_batches = 0
        for X_batch_val, y_batch_val in val_dataset:
            loss = test_step_amrl(model, loss_function, X_batch_val, y_batch_val, val_metrics)
            total_val_loss += loss.numpy()
            num_val_batches += 1

        avg_val_loss = total_val_loss / num_val_batches

        # Get metric results
        train_acc = train_metrics['accuracy'].result().numpy()
        train_prec = train_metrics['precision'].result().numpy()
        train_recall = train_metrics['recall'].result().numpy()
        train_auc_roc = train_metrics['auc_roc'].result().numpy()
        train_auc_pr = train_metrics['auc_pr'].result().numpy()

        val_acc = val_metrics['accuracy'].result().numpy()
        val_prec = val_metrics['precision'].result().numpy()
        val_recall = val_metrics['recall'].result().numpy()
        val_auc_roc = val_metrics['auc_roc'].result().numpy()
        val_auc_pr = val_metrics['auc_pr'].result().numpy()

        # Append metrics to history for plotting
        history['loss'].append(avg_train_loss)
        history['accuracy'].append(train_acc)
        history['precision'].append(train_prec)
        history['recall'].append(train_recall)
        history['auc_roc'].append(train_auc_roc)
        history['auc_pr'].append(train_auc_pr)
        history['val_loss'].append(avg_val_loss)
        history['val_accuracy'].append(val_acc)
        history['val_precision'].append(val_prec)
        history['val_recall'].append(val_recall)
        history['val_auc_roc'].append(val_auc_roc)
        history['val_auc_pr'].append(val_auc_pr)


        print(f"Train Loss: {avg_train_loss:.4f}, Train Acc: {train_acc:.4f}, Train Prec: {train_prec:.4f}, Train Recall: {train_recall:.4f}, Train AUC-ROC: {train_auc_roc:.4f}, Train AUC-PR: {train_auc_pr:.4f}")
        print(f"Val Loss: {avg_val_loss:.4f}, Val Acc: {val_acc:.4f}, Val Prec: {val_prec:.4f}, Val Recall: {val_recall:.4f}, Val AUC-ROC: {val_auc_roc:.4f}, Val AUC-PR: {val_auc_pr:.4f}")

        # Print the current values of the trainable loss parameters
        if isinstance(loss_function, AMRLLossTF):
            learned_params = [loss_function.m1.numpy(), loss_function.m0.numpy(), loss_function.lambda1.numpy(), loss_function.lambda0.numpy()]
            print(f"  AMRL Learned Params: m1={learned_params[0]:.4f}, m0={learned_params[1]:.4f}, lambda1={learned_params[2]:.4f}, lambda0={learned_params[3]:.4f}")

        # --- Custom Stopping Condition: Stop if Train Loss becomes negative ---
        if avg_train_loss < 0:
            print(f"Stopping training: Train Loss became negative at epoch {epoch+1}.")
            # Load weights from the previous epoch (if saved)
            if os.path.exists(model_weights_path):
                print("Restoring model weights from previous epoch...")
                model.load_weights(model_weights_path)
            if os.path.exists(loss_weights_path):
                print("Restoring loss parameters from previous epoch...")
                # Assuming the order of saving is [m1, m0, lambda1, lambda0]
                prev_loss_params = np.load(loss_weights_path)
                loss_function.m1.assign(prev_loss_params[0])
                loss_function.m0.assign(prev_loss_params[1])
                loss_function.lambda1.assign(prev_loss_params[2])
                loss_function.lambda0.assign(prev_loss_params[3])
                print(f"  Restored AMRL Params: m1={loss_function.m1.numpy():.4f}, m0={loss_function.m0.numpy():.4f}, lambda1={loss_function.lambda1.numpy():.4f}, lambda0={loss_function.lambda0.numpy():.4f}")
            break # Exit the training loop


        # --- Early Stopping Check (using validation loss) ---
        # Only perform early stopping based on validation loss if train loss is not negative
        if avg_val_loss < best_val_loss - early_stopping.min_delta:
            best_val_loss = avg_val_loss
            epochs_without_improvement = 0
            # Save current weights if validation loss improved
            model.save_weights(model_weights_path)
            # Save loss function trainable variables
            np.save(loss_weights_path, learned_params) # Save the current learned parameters
            #print("Saved model and loss weights.")

        else:
            epochs_without_improvement += 1
            if epochs_without_improvement >= early_stopping.patience:
                print(f"Early stopping triggered based on validation loss after {epoch+1} epochs.")
                # Restore best weights if early stopping is triggered
                if os.path.exists(model_weights_path):
                    print("Restoring model weights from best validation epoch...")
                    model.load_weights(model_weights_path)
                if os.path.exists(loss_weights_path):
                    print("Restoring loss parameters from best validation epoch...")
                     # Assuming the order of saving is [m1, m0, lambda1, lambda0]
                    best_loss_params = np.load(loss_weights_path)
                    loss_function.m1.assign(best_loss_params[0])
                    loss_function.m0.assign(best_loss_params[1])
                    loss_function.lambda1.assign(best_loss_params[2])
                    loss_function.lambda0.assign(best_loss_params[3])
                    print(f"  Restored AMRL Params: m1={loss_function.m1.numpy():.4f}, m0={loss_function.m0.numpy():.4f}, lambda1={loss_function.lambda1.numpy():.4f}, lambda0={loss_function.lambda0.numpy():.4f}")

                break # Exit the training loop

    return history # Return the history dictionary

In [ ]:
# Custom train step for PGFL with gradient clipping
# Removed @tf.function decorator
def train_step_pgfl(model, loss_function, optimizer, X_batch, y_batch, clip_norm):
    with tf.GradientTape() as tape:
        y_pred_logits = model(X_batch, training=True) # Set training=True for dropout
        loss = loss_function(y_batch, y_pred_logits) # loss.call calculates loss

    # Update moving averages *after* calculating the loss and outside the gradient tape
    # This ensures the update uses the gradients from the loss calculation
    y_pred_probs = tf.sigmoid(y_pred_logits)
    if isinstance(loss_function, PGFLossTF): # Only update for PGFLossTF
        loss_function.update_confidence_moving_averages(y_batch, y_pred_probs)


    # Get gradients for model's trainable variables AND loss function's trainable variable (alpha)
    trainable_variables = model.trainable_variables + loss_function.trainable_variables if hasattr(loss_function, 'trainable_variables') else model.trainable_variables
    gradients = tape.gradient(loss, trainable_variables)

    # Add a check for None gradients and filter
    if any(grad is None for grad in gradients):
        none_grads = [var.name for grad, var in zip(gradients, trainable_variables) if grad is None]
        # print(f"Warning: Gradients are None for variables: {none_grads}") # Keep this for debugging if needed
        grads_and_vars = [(grad, var) for grad, var in zip(gradients, trainable_variables) if grad is not None]
        if not grads_and_vars:
            raise ValueError("All gradients are None. Cannot apply gradients.")
    else:
        grads_and_vars = list(zip(gradients, trainable_variables))


    # Apply gradient clipping
    clipped_gradients, _ = tf.clip_by_global_norm([g for g, v in grads_and_vars], clip_norm)

    # Apply gradients
    optimizer.apply_gradients(zip(clipped_gradients, [v for g, v in grads_and_vars]))


    return loss

# Custom test step for PGFL (used for validation)
# Removed @tf.function decorator
def test_step_pgfl(model, loss_function, X_batch, y_batch, metrics):
    y_pred_logits = model(X_batch, training=False) # Set training=False for dropout
    loss = loss_function(y_batch, y_pred_logits) # loss.call updates MC0/MC1 (can stay in call for test, or move out)

    # Update metrics
    y_pred_probs = tf.sigmoid(y_pred_logits)
    metrics['accuracy'].update_state(y_batch, y_pred_probs)
    metrics['precision'].update_state(y_batch, y_pred_probs)
    metrics['recall'].update_state(y_batch, y_pred_probs)
    metrics['auc_roc'].update_state(y_batch, y_pred_probs)
    metrics['auc_pr'].update_state(y_batch, y_pred_probs)

    return loss

# Custom training loop for PGFL
def custom_train_loop_pgfl(model, loss_function, optimizer, train_dataset, val_dataset, epochs, early_stopping, clip_norm):
    best_val_loss = float('inf')
    epochs_without_improvement = 0
    # Define paths for saving temporary weights for both models
    model_weights_path = 'temp_model_weights_pgfl.weights.h5' # Separate paths for PGFL
    loss_weights_path = 'temp_loss_weights_pgfl.npy' # Separate paths for PGFL

    history = {
        'loss': [], 'accuracy': [], 'precision': [], 'recall': [], 'auc_roc': [], 'auc_pr': [],
        'val_loss': [], 'val_accuracy': [], 'val_precision': [], 'val_recall': [], 'val_auc_roc': [], 'val_auc_pr': []
    }

    train_metrics = {
        'accuracy': BinaryAccuracy(threshold=0.5), 'precision': Precision(thresholds=0.5),
        'recall': Recall(thresholds=0.5), 'auc_roc': AUC(), 'auc_pr': AUC(curve='PR')
    }
    val_metrics = {
        'accuracy': BinaryAccuracy(threshold=0.5), 'precision': Precision(thresholds=0.5),
        'recall': Recall(thresholds=0.5), 'auc_roc': AUC(), 'auc_pr': AUC(curve='PR')
    }


    for epoch in range(epochs):
        print(f"\nEpoch {epoch+1}/{epochs}")

        for metric in train_metrics.values(): metric.reset_state()
        for metric in val_metrics.values(): metric.reset_state()
        # Reset loss function's internal metrics (MC0, MC1) at the start of each epoch?
        # Or let them accumulate? The description suggests moving averages over time,
        # so perhaps don't reset per epoch, but per file. The re-initialization per file handles this.
        # loss_function.reset_metrics() # Only if resetting per epoch is desired


        total_train_loss = 0
        num_train_batches = 0
        for i, (X_batch, y_batch) in enumerate(train_dataset): # Add enumerate to get batch index
            loss = train_step_pgfl(model, loss_function, optimizer, X_batch, y_batch, clip_norm)
            total_train_loss += loss.numpy()
            num_train_batches += 1
            # print(f"  Batch {i}: Loss = {loss.numpy():.4f}") # Print loss per batch


            # Update train metrics manually
            y_pred_logits = model(X_batch, training=False)
            y_pred_probs = tf.sigmoid(y_pred_logits)
            train_metrics['accuracy'].update_state(y_batch, y_pred_probs)
            train_metrics['precision'].update_state(y_batch, y_pred_probs)
            train_metrics['recall'].update_state(y_batch, y_pred_probs)
            train_metrics['auc_roc'].update_state(y_batch, y_pred_probs)
            train_metrics['auc_pr'].update_state(y_batch, y_pred_probs)


        avg_train_loss = total_train_loss / num_train_batches

        total_val_loss = 0
        num_val_batches = 0
        for X_batch_val, y_batch_val in val_dataset:
            loss = test_step_pgfl(model, loss_function, X_batch_val, y_batch_val, val_metrics)
            total_val_loss += loss.numpy()
            num_val_batches += 1

        avg_val_loss = total_val_loss / num_val_batches

        train_acc = train_metrics['accuracy'].result().numpy()
        train_prec = train_metrics['precision'].result().numpy()
        train_recall = train_metrics['recall'].result().numpy()
        train_auc_roc = train_metrics['auc_roc'].result().numpy()
        train_auc_pr = train_metrics['auc_pr'].result().numpy()

        val_acc = val_metrics['accuracy'].result().numpy()
        val_prec = val_metrics['precision'].result().numpy()
        val_recall = val_metrics['recall'].result().numpy()
        val_auc_roc = val_metrics['auc_roc'].result().numpy()
        val_auc_pr = val_metrics['auc_pr'].result().numpy()

        history['loss'].append(avg_train_loss)
        history['accuracy'].append(train_acc)
        history['precision'].append(train_prec)
        history['recall'].append(train_recall)
        history['auc_roc'].append(train_auc_roc)
        history['auc_pr'].append(train_auc_pr)
        history['val_loss'].append(avg_val_loss)
        history['val_accuracy'].append(val_acc)
        history['val_precision'].append(val_prec)
        history['val_recall'].append(val_recall)
        history['val_auc_roc'].append(val_auc_roc)
        history['val_auc_pr'].append(val_auc_pr)


        print(f"Train Loss: {avg_train_loss:.4f}, Train Acc: {train_acc:.4f}, Train Prec: {train_prec:.4f}, Train Recall: {train_recall:.4f}, Train AUC-ROC: {train_auc_roc:.4f}, Train AUC-PR: {train_auc_pr:.4f}")
        print(f"Val Loss: {avg_val_loss:.4f}, Val Acc: {val_acc:.4f}, Val Prec: {val_prec:.4f}, Val Recall: {val_recall:.4f}, Val AUC-ROC: {val_auc_roc:.4f}, Val AUC-PR: {val_auc_pr:.4f}")

        # Print the current value of the trainable alpha and moving averages
        if isinstance(loss_function, PGFLossTF):
             print(f"  PGFL Learned Alpha: {loss_function.alpha.numpy():.4f}, MC0: {loss_function.mc0.numpy():.4f}, MC1: {loss_function.mc1.numpy():.4f}")


        # --- Early Stopping Check (using validation loss) ---
        if avg_val_loss < best_val_loss - early_stopping.min_delta:
            best_val_loss = avg_val_loss
            epochs_without_improvement = 0
            # Save current weights if validation loss improved
            model.save_weights(model_weights_path)
            # Save loss function trainable variables (alpha) and moving averages
            np.save(loss_weights_path, [loss_function.alpha.numpy(), loss_function.mc0.numpy(), loss_function.mc1.numpy()])
            #print("Saved model and PGFL loss parameters.")

        else:
            epochs_without_improvement += 1
            if epochs_without_improvement >= early_stopping.patience:
                print(f"Early stopping triggered based on validation loss after {epoch+1} epochs.")
                # Restore best weights if early stopping is triggered
                if os.path.exists(model_weights_path):
                    print("Restoring model weights from best validation epoch...")
                    model.load_weights(model_weights_path)
                if os.path.exists(loss_weights_path):
                    print("Restoring PGFL loss parameters from best validation epoch...")
                    best_loss_params = np.load(loss_weights_path)
                    loss_function.alpha.assign(best_loss_params[0])
                    loss_function.mc0.assign(best_loss_params[1])
                    loss_function.mc1.assign(best_loss_params[2])
                    print(f"  Restored PGFL Params: Alpha={loss_function.alpha.numpy():.4f}, MC0={loss_function.mc0.numpy():.4f}, MC1={loss_function.mc1.numpy():.4f}")

                break # Exit the training loop

    return history # Return the history dictionary


In [ ]:
# Custom train step for GAN (Generator and Discriminator)
# Removed @tf.function decorator
def train_step_gan(generator, discriminator, generator_optimizer, discriminator_optimizer,
                   classification_loss_fn, X_batch, y_batch, clip_norm, gan_lambda=0.1):
    """
    Performs one training step for the GAN setup.
    Updates both the generator (classifier) and the discriminator.
    """
    # --- Train Discriminator ---
    # Train discriminator DISC_TRAIN_RATIO times for each generator step
    for _ in range(DISC_TRAIN_RATIO):
        with tf.GradientTape() as disc_tape:
            # Get generator's prediction (logits or probs) for the real data batch
            # Use training=False here for the generator during discriminator training
            # to get deterministic output, unless dropout is specifically desired.
            # Let's use probabilities for input to discriminator as it's a probability prediction.
            y_pred_logits_fake = generator(X_batch, training=False)
            y_pred_probs_fake = tf.sigmoid(y_pred_logits_fake)

            # Add noise to discriminator inputs if ADD_NOISE is True
            if ADD_NOISE:
                 y_batch_noisy = y_batch + tf.random.normal(tf.shape(y_batch), stddev=NOISE_STDDEV)
                 y_batch_noisy = tf.clip_by_value(y_batch_noisy, 0., 1.) # Clip to valid probability range

                 y_pred_probs_fake_noisy = y_pred_probs_fake + tf.random.normal(tf.shape(y_pred_probs_fake), stddev=NOISE_STDDEV)
                 y_pred_probs_fake_noisy = tf.clip_by_value(y_pred_probs_fake_noisy, 0., 1.)


                 # Prepare inputs for the discriminator with noise
                 # Real samples: [X_batch, y_batch_noisy]
                 # Fake samples: [X_batch, y_pred_probs_fake_noisy]
                 disc_input_real = [X_batch, y_batch_noisy]
                 disc_input_fake = [X_batch, y_pred_probs_fake_noisy]

            else:
                 # Prepare inputs for the discriminator without noise
                 # Real samples: [X_batch, y_batch]
                 # Fake samples: [X_batch, y_pred_probs_fake]
                 disc_input_real = [X_batch, y_batch]
                 disc_input_fake = [X_batch, y_pred_probs_fake]


            # Discriminator output for real samples (target is 1)
            disc_output_real = discriminator(disc_input_real, training=True)
            # Discriminator output for fake samples (target is 0)
            disc_output_fake = discriminator(disc_input_fake, training=True)

            # Calculate discriminator loss
            # Loss for real samples (should predict 1)
            real_loss = tf.keras.losses.binary_crossentropy(tf.ones_like(disc_output_real), disc_output_real)
            # Loss for fake samples (should predict 0)
            fake_loss = tf.keras.losses.binary_crossentropy(tf.zeros_like(disc_output_fake), disc_output_fake)

            # Total discriminator loss
            disc_loss = tf.reduce_mean(real_loss) + tf.reduce_mean(fake_loss)

        # Get and apply gradients for the discriminator
        disc_gradients = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
        # Apply gradient clipping to discriminator
        clipped_disc_gradients, _ = tf.clip_by_global_norm(disc_gradients, clip_norm)

        discriminator_optimizer.apply_gradients(zip(clipped_disc_gradients, discriminator.trainable_variables))


    # --- Train Generator ---
    # We train the generator to make the discriminator classify fake samples as real (target is 1)
    # Keep discriminator trainable=False during generator training
    discriminator.trainable = False
    with tf.GradientTape() as gen_tape:
        # Get generator's prediction (logits) - Use training=True for dropout
        y_pred_logits_gen = generator(X_batch, training=True)
        y_pred_probs_gen = tf.sigmoid(y_pred_logits_gen)

        # Calculate generator's classification loss (e.g., BCE or other imbalanced loss)
        # This is the standard loss the classifier would use without the GAN component.
        # Use from_logits=True if classification_loss_fn expects logits
        gen_classification_loss = classification_loss_fn(y_batch, y_pred_logits_gen)

        # Get discriminator's output for the generator's fake samples
        # Now the generator wants the discriminator to think these are real (target is 1)
        # Add noise to generator's output before feeding to discriminator if ADD_NOISE is True
        if ADD_NOISE:
             y_pred_probs_gen_noisy = y_pred_probs_gen + tf.random.normal(tf.shape(y_pred_probs_gen), stddev=NOISE_STDDEV)
             y_pred_probs_gen_noisy = tf.clip_by_value(y_pred_probs_gen_noisy, 0., 1.)
             disc_output_for_gen = discriminator([X_batch, y_pred_probs_gen_noisy], training=False) # Use training=False for discriminator here
        else:
             disc_output_for_gen = discriminator([X_batch, y_pred_probs_gen], training=False) # Use training=False for discriminator here


        # Calculate generator's adversarial loss
        # The generator wants the discriminator's output for fake samples to be close to 1.
        gen_adversarial_loss = tf.keras.losses.binary_crossentropy(tf.ones_like(disc_output_for_gen), disc_output_for_gen)

        # Total generator loss: Combination of classification loss and adversarial loss
        gen_total_loss = gen_classification_loss + gan_lambda * gen_adversarial_loss # gan_lambda is a weighting factor

    # Get and apply gradients for the generator
    gen_gradients = gen_tape.gradient(gen_total_loss, generator.trainable_variables)
     # Apply gradient clipping to generator
    clipped_gen_gradients, _ = tf.clip_by_global_norm(gen_gradients, clip_norm)

    generator_optimizer.apply_gradients(zip(clipped_gen_gradients, generator.trainable_variables))

    # Set discriminator trainable back to True
    discriminator.trainable = True

    # Return losses and potentially metrics
    return gen_total_loss, disc_loss, gen_classification_loss, gen_adversarial_loss

# Custom Training Loop for GAN
def custom_train_loop_gan(generator, discriminator, generator_optimizer, discriminator_optimizer,
                          classification_loss_fn, train_dataset, val_dataset, epochs,
                          early_stopping, clip_norm, gan_lambda=0.1):

    best_val_loss = float('inf')
    epochs_without_improvement = 0
    # Define paths for saving temporary weights for both models
    generator_weights_path = 'temp_generator_weights.weights.h5' # Separate paths for PGFL
    discriminator_weights_path = 'temp_discriminator_weights.weights.h5' # Separate paths for PGFL


    history = {
        'gen_loss': [], 'disc_loss': [], 'gen_clf_loss': [], 'gen_adv_loss': [],
        'accuracy': [], 'precision': [], 'recall': [], 'auc_roc': [], 'auc_pr': [],
        'val_loss': [], 'val_accuracy': [], 'val_precision': [], 'val_recall': [], 'val_auc_roc': [], 'val_auc_pr': []
    }

    # Initialize metrics for tracking generator performance within the custom loop
    train_metrics = {
        'accuracy': BinaryAccuracy(threshold=0.5), 'precision': Precision(thresholds=0.5),
        'recall': Recall(thresholds=0.5), 'auc_roc': AUC(), 'auc_pr': AUC(curve='PR')
    }
    val_metrics = {
        'accuracy': BinaryAccuracy(threshold=0.5), 'precision': Precision(thresholds=0.5),
        'recall': Recall(thresholds=0.5), 'auc_roc': AUC(), 'auc_pr': AUC(curve='PR')
    }


    for epoch in range(epochs):
        # print(f"\nEpoch {epoch+1}/{epochs}") # Reduced verbosity

        # Reset metrics at the start of each epoch
        for metric in train_metrics.values(): metric.reset_state()
        for metric in val_metrics.values(): val_metrics[metric.name].reset_state()


        total_gen_loss = 0
        total_disc_loss = 0
        total_gen_clf_loss = 0
        total_gen_adv_loss = 0
        num_train_batches = 0

        for X_batch, y_batch in train_dataset:
            gen_loss, disc_loss, gen_clf_loss, gen_adv_loss = train_step_gan(
                generator, discriminator, generator_optimizer, discriminator_optimizer,
                classification_loss_fn, X_batch, y_batch, clip_norm, gan_lambda
            )
            total_gen_loss += gen_loss.numpy()
            total_disc_loss += disc_loss.numpy()
            total_gen_clf_loss += gen_clf_loss.numpy()
            total_gen_adv_loss += gen_adv_loss.numpy()
            num_train_batches += 1

            # Update train metrics for the generator (classifier)
            y_pred_logits = generator(X_batch, training=False) # Use training=False for deterministic predictions
            y_pred_probs = tf.sigmoid(y_pred_logits)
            train_metrics['accuracy'].update_state(y_batch, y_pred_probs)
            train_metrics['precision'].update_state(y_batch, y_pred_probs)
            train_metrics['recall'].update_state(y_batch, y_pred_probs)
            train_metrics['auc_roc'].update_state(y_batch, y_pred_probs)
            train_metrics['auc_pr'].update_state(y_batch, y_pred_probs)


        avg_gen_loss = total_gen_loss / num_train_batches
        avg_disc_loss = total_disc_loss / num_train_batches
        avg_gen_clf_loss = total_gen_clf_loss / num_train_batches
        avg_gen_adv_loss = total_gen_adv_loss / num_train_batches


        # Evaluate generator on validation set
        total_val_loss = 0 # This will track the generator's classification loss on validation
        num_val_batches = 0
        for X_batch_val, y_batch_val in val_dataset:
             # Calculate classification loss on validation
            y_pred_logits_val = generator(X_batch_val, training=False)
            val_classification_loss = classification_loss_fn(y_batch_val, y_pred_logits_val)
            total_val_loss += val_classification_loss.numpy()
            num_val_batches += 1

            # Update val metrics for the generator
            y_pred_probs_val = tf.sigmoid(y_pred_logits_val)
            val_metrics['accuracy'].update_state(y_batch_val, y_pred_probs_val)
            val_metrics['precision'].update_state(y_batch_val, y_pred_probs_val)
            val_metrics['recall'].update_state(y_batch_val, y_pred_probs_val)
            val_metrics['auc_roc'].update_state(y_batch_val, y_pred_probs_val)
            val_metrics['auc_pr'].update_state(y_batch_val, y_pred_probs_val)


        avg_val_loss = total_val_loss / num_val_batches # This is the average classification loss on validation


        # Get metric results
        train_acc = train_metrics['accuracy'].result().numpy()
        train_prec = train_metrics['precision'].result().numpy()
        train_recall = train_metrics['recall'].result().numpy()
        train_auc_roc = train_metrics['auc_roc'].result().numpy()
        train_auc_pr = train_metrics['auc_pr'].result().numpy()

        val_acc = val_metrics['accuracy'].result().numpy()
        val_prec = val_metrics['precision'].result().numpy()
        val_recall = val_metrics['recall'].result().numpy()
        val_auc_roc = val_metrics['auc_roc'].result().numpy()
        val_auc_pr = val_metrics['auc_pr'].result().numpy()

        # Append metrics to history for plotting
        history['gen_loss'].append(avg_gen_loss)
        history['disc_loss'].append(avg_disc_loss)
        history['gen_clf_loss'].append(avg_gen_clf_loss)
        history['gen_adv_loss'].append(avg_gen_adv_loss)
        history['accuracy'].append(train_acc)
        history['precision'].append(train_prec)
        history['recall'].append(train_recall)
        history['auc_roc'].append(train_auc_roc)
        history['auc_pr'].append(train_auc_pr)
        history['val_loss'].append(avg_val_loss) # Validation Classification Loss
        history['val_accuracy'].append(val_acc)
        history['val_precision'].append(val_prec)
        history['val_recall'].append(val_recall)
        history['val_auc_roc'].append(val_auc_roc)
        history['val_auc_pr'].append(val_auc_pr)


        # print(f"  Train Gen Loss: {avg_gen_loss:.4f}, Train Disc Loss: {avg_disc_loss:.4f}, Train Gen CLF Loss: {avg_gen_clf_loss:.4f}, Train Gen ADV Loss: {avg_gen_adv_loss:.4f}") # Reduced verbosity
        # print(f"  Train Acc: {train_acc:.4f}, Train Prec: {train_prec:.4f}, Train Recall: {train_recall:.4f}, Train AUC-ROC: {train_auc_roc:.4f}, Train AUC-PR: {train_auc_pr:.4f}") # Reduced verbosity
        # print(f"  Val Loss: {avg_val_loss:.4f}, Val Acc: {val_acc:.4f}, Val Prec: {val_prec:.4f}, Val Recall: {val_recall:.4f}, Val AUC-ROC: {val_auc_roc:.4f}, Val AUC-PR: {val_auc_pr:.4f}") # Reduced verbosity


        # --- Early Stopping Check (using validation classification loss) ---
        if avg_val_loss < best_val_loss - early_stopping.min_delta:
            best_val_loss = avg_val_loss
            epochs_without_improvement = 0
            # Save current weights if validation loss improved
            generator.save_weights(generator_weights_path)
            discriminator.save_weights(discriminator_weights_path)
            #print("Saved generator and discriminator weights.")

        else:
            epochs_without_improvement += 1
            if epochs_without_improvement >= early_stopping.patience:
                print(f"Early stopping triggered based on validation loss after {epoch+1} epochs.")
                # Restore best weights if early stopping is triggered
                if os.path.exists(generator_weights_path):
                    print("Restoring generator weights from best validation epoch...")
                    generator.load_weights(generator_weights_path)
                if os.path.exists(discriminator_weights_path):
                    print("Restoring discriminator weights from best validation epoch...")
                    discriminator.load_weights(discriminator_weights_path)

                break # Exit the training loop

    return history # Return the history dictionary

In [ ]:
from tensorflow.keras import Model, layers # Import Model and layers

class Discriminator(Model):
    """
    Discriminator network for GAN-based imbalanced classification.
    Takes features (X) and classifier output (logits or probabilities) as input
    and outputs the probability that the sample is 'real' (from the true distribution).
    """
    def __init__(self, input_dim_X, input_dim_Y):
        super(Discriminator, self).__init__()
        # The discriminator needs to handle both the features X and the classifier's output Y.
        # We can concatenate them or process them separately and combine later.
        # A simple approach is concatenation after potential initial processing.
        # Let's start with concatenating the raw features and the classifier's single output (logit or probability).
        # Total input dimension will be input_dim_X + input_dim_Y (which is 1 for binary classification output).
        combined_input_dim = input_dim_X + input_dim_Y

        self.fc1 = layers.Dense(64, activation='relu', input_shape=(combined_input_dim,))
        self.dropout1 = layers.Dropout(0.3)
        self.fc2 = layers.Dense(32, activation='relu')
        self.dropout2 = layers.Dropout(0.3)
        # Output layer: single unit with sigmoid activation for real/fake probability
        self.output_layer = layers.Dense(1, activation='sigmoid')

    def call(self, inputs, training=False):
        # inputs is expected to be a tuple or list: [X, Y_classifier_output]
        X_input, Y_classifier_output = inputs

        # Ensure Y_classifier_output has a shape that can be concatenated (e.g., reshape if needed)
        # For binary classification logits/probs, it should be (batch_size, 1)
        if tf.rank(Y_classifier_output) == 0:
             Y_classifier_output = tf.reshape(Y_classifier_output, (-1, 1))
        elif tf.rank(Y_classifier_output) == 1:
             Y_classifier_output = tf.expand_dims(Y_classifier_output, axis=-1)


        # Concatenate features and classifier output
        combined_input = tf.concat([X_input, Y_classifier_output], axis=-1)

        x = self.fc1(combined_input)
        x = self.dropout1(x, training=training)
        x = self.fc2(x)
        x = self.dropout2(x, training=training)
        discriminator_output = self.output_layer(x)

        return discriminator_output

In [ ]:
# --- Custom Training Step for GAN (Generator and Discriminator) ---
# This function will perform one training iteration, including updating both
# the generator (SimpleMLP_TF) and the discriminator.

# We need separate optimizers for the generator and discriminator.
# Let's define them outside this function.

# We also need a base classification loss (e.g., BCE) for the generator.
# Let's define that outside as well.

def train_step_gan(generator, discriminator, generator_optimizer, discriminator_optimizer,
                   classification_loss_fn, X_batch, y_batch, clip_norm, gan_lambda=0.1):
    """
    Performs one training step for the GAN setup.
    Updates both the generator (classifier) and the discriminator.
    """
    # --- Train Discriminator ---
    with tf.GradientTape() as disc_tape:
        # Get generator's prediction (logits or probs) for the real data batch
        # Use training=False here for the generator during discriminator training
        # to get deterministic output, unless dropout is specifically desired.
        # Let's use logits for consistency with classification loss.
        y_pred_logits_fake = generator(X_batch, training=False)
        y_pred_probs_fake = tf.sigmoid(y_pred_logits_fake)

        # Prepare inputs for the discriminator
        # Real samples: [X_batch, y_batch]
        # Fake samples: [X_batch, y_pred_probs_fake] (using probabilities for fake)

        # Discriminator output for real samples (target is 1)
        disc_output_real = discriminator([X_batch, y_batch], training=True)
        # Discriminator output for fake samples (target is 0)
        disc_output_fake = discriminator([X_batch, y_pred_probs_fake], training=True)

        # Calculate discriminator loss
        # Loss for real samples (should predict 1)
        real_loss = tf.keras.losses.binary_crossentropy(tf.ones_like(disc_output_real), disc_output_real)
        # Loss for fake samples (should predict 0)
        fake_loss = tf.keras.losses.binary_crossentropy(tf.zeros_like(disc_output_fake), disc_output_fake)

        # Total discriminator loss
        disc_loss = tf.reduce_mean(real_loss) + tf.reduce_mean(fake_loss)

    # Get and apply gradients for the discriminator
    disc_gradients = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    # Apply gradient clipping to discriminator
    clipped_disc_gradients, _ = tf.clip_by_global_norm(disc_gradients, clip_norm)

    discriminator_optimizer.apply_gradients(zip(clipped_disc_gradients, discriminator.trainable_variables))

    # --- Train Generator ---
    # We train the generator to make the discriminator classify fake samples as real (target is 1)
    with tf.GradientTape() as gen_tape:
        # Get generator's prediction (logits) - Use training=True for dropout
        y_pred_logits_gen = generator(X_batch, training=True)
        y_pred_probs_gen = tf.sigmoid(y_pred_logits_gen)

        # Calculate generator's classification loss (e.g., BCE or other imbalanced loss)
        # This is the standard loss the classifier would use without the GAN component.
        # Use from_logits=True if classification_loss_fn expects logits
        gen_classification_loss = classification_loss_fn(y_batch, y_pred_logits_gen)

        # Get discriminator's output for the generator's fake samples
        # Now the generator wants the discriminator to think these are real (target is 1)
        disc_output_for_gen = discriminator([X_batch, y_pred_probs_gen], training=False) # Use training=False for discriminator here

        # Calculate generator's adversarial loss
        # The generator wants the discriminator's output for fake samples to be close to 1.
        gen_adversarial_loss = tf.keras.losses.binary_crossentropy(tf.ones_like(disc_output_for_gen), disc_output_for_gen)

        # Total generator loss: Combination of classification loss and adversarial loss
        gen_total_loss = gen_classification_loss + gan_lambda * gen_adversarial_loss # gan_lambda is a weighting factor

    # Get and apply gradients for the generator
    gen_gradients = gen_tape.gradient(gen_total_loss, generator.trainable_variables)
     # Apply gradient clipping to generator
    clipped_gen_gradients, _ = tf.clip_by_global_norm(gen_gradients, clip_norm)

    generator_optimizer.apply_gradients(zip(clipped_gen_gradients, generator.trainable_variables))

    # Return losses and potentially metrics
    # Ensure losses are returned as scalar numpy values for easier accumulation
    return gen_total_loss.numpy(), disc_loss.numpy(), gen_classification_loss.numpy(), gen_adversarial_loss.numpy()

# --- Custom Training Loop for GAN ---
# This loop will orchestrate the training steps, including early stopping.
# It needs to track metrics for both generator and discriminator if desired,
# but the primary focus is on the generator's performance on the classification task.

def custom_train_loop_gan(generator, discriminator, generator_optimizer, discriminator_optimizer,
                          classification_loss_fn, train_dataset, val_dataset, epochs,
                          early_stopping, clip_norm, gan_lambda=0.1):

    best_val_loss = float('inf')
    epochs_without_improvement = 0
    # Define paths for saving temporary weights for both models
    generator_weights_path = 'temp_generator_weights.weights.h5'
    discriminator_weights_path = 'temp_discriminator_weights.weights.h5'


    history = {
        'gen_loss': [], 'disc_loss': [], 'gen_clf_loss': [], 'gen_adv_loss': [],
        'accuracy': [], 'precision': [], 'recall': [], 'auc_roc': [], 'auc_pr': [],
        'val_loss': [], 'val_accuracy': [], 'val_precision': [], 'val_recall': [], 'val_auc_roc': [], 'val_auc_pr': []
    }

    # Initialize metrics for tracking generator performance within the custom loop
    train_metrics = {
        'accuracy': BinaryAccuracy(threshold=0.5), 'precision': Precision(thresholds=0.5),
        'recall': Recall(thresholds=0.5), 'auc_roc': AUC(), 'auc_pr': AUC(curve='PR')
    }
    val_metrics = {
        'accuracy': BinaryAccuracy(threshold=0.5), 'precision': Precision(thresholds=0.5),
        'recall': Recall(thresholds=0.5), 'auc_roc': AUC(), 'auc_pr': AUC(curve='PR')
    }


    for epoch in range(epochs):
        print(f"\nEpoch {epoch+1}/{epochs}")

        # Reset metrics at the start of each epoch
        for metric in train_metrics.values(): metric.reset_state()
        for metric in val_metrics.values(): val_metrics[metric.name].reset_state()


        total_gen_loss = 0
        total_disc_loss = 0
        total_gen_clf_loss = 0
        total_gen_adv_loss = 0
        num_train_batches = 0

        for X_batch, y_batch in train_dataset:
            gen_loss, disc_loss, gen_clf_loss, gen_adv_loss = train_step_gan(
                generator, discriminator, generator_optimizer, discriminator_optimizer,
                classification_loss_fn, X_batch, y_batch, clip_norm, gan_lambda
            )
            total_gen_loss += gen_loss
            total_disc_loss += disc_loss
            total_gen_clf_loss += gen_clf_loss
            total_gen_adv_loss += gen_adv_loss
            num_train_batches += 1

            # Update train metrics for the generator (classifier)
            y_pred_logits = generator(X_batch, training=False) # Use training=False for deterministic predictions
            y_pred_probs = tf.sigmoid(y_pred_logits)
            train_metrics['accuracy'].update_state(y_batch, y_pred_probs)
            train_metrics['precision'].update_state(y_batch, y_pred_probs)
            train_metrics['recall'].update_state(y_batch, y_pred_probs)
            train_metrics['auc_roc'].update_state(y_batch, y_pred_probs)
            train_metrics['auc_pr'].update_state(y_batch, y_pred_probs)


        avg_gen_loss = total_gen_loss / num_train_batches
        avg_disc_loss = total_disc_loss / num_train_batches
        avg_gen_clf_loss = total_gen_clf_loss / num_train_batches
        avg_gen_adv_loss = total_gen_adv_loss / num_train_batches


        # Evaluate generator on validation set
        total_val_loss = 0 # This will track the generator's classification loss on validation
        num_val_batches = 0
        for X_batch_val, y_batch_val in val_dataset:
             # Calculate classification loss on validation
            y_pred_logits_val = generator(X_batch_val, training=False)
            val_classification_loss = classification_loss_fn(y_batch_val, y_pred_logits_val)
            total_val_loss += val_classification_loss.numpy()
            num_val_batches += 1

            # Update val metrics for the generator
            y_pred_probs_val = tf.sigmoid(y_pred_logits_val)
            val_metrics['accuracy'].update_state(y_batch_val, y_pred_probs_val)
            val_metrics['precision'].update_state(y_batch_val, y_pred_probs_val)
            val_metrics['recall'].update_state(y_batch_val, y_pred_probs_val)
            val_metrics['auc_roc'].update_state(y_batch_val, y_pred_probs_val)
            val_metrics['auc_pr'].update_state(y_batch_val, y_pred_probs_val)


        avg_val_loss = total_val_loss / num_val_batches # This is the average classification loss on validation


        # Get metric results
        train_acc = train_metrics['accuracy'].result().numpy()
        train_prec = train_metrics['precision'].result().numpy()
        train_recall = train_metrics['recall'].result().numpy()
        train_auc_roc = train_metrics['auc_roc'].result().numpy()
        train_auc_pr = train_metrics['auc_pr'].result().numpy()

        val_acc = val_metrics['accuracy'].result().numpy()
        val_prec = val_metrics['precision'].result().numpy()
        val_recall = val_metrics['recall'].result().numpy()
        val_auc_roc = val_metrics['auc_roc'].result().numpy()
        val_auc_pr = val_metrics['auc_pr'].result().numpy()

        # Append metrics to history for plotting
        history['gen_loss'].append(avg_gen_loss)
        history['disc_loss'].append(avg_disc_loss)
        history['gen_clf_loss'].append(avg_gen_clf_loss)
        history['gen_adv_loss'].append(avg_gen_adv_loss)
        history['accuracy'].append(train_acc)
        history['precision'].append(train_prec)
        history['recall'].append(train_recall)
        history['auc_roc'].append(train_auc_roc)
        history['auc_pr'].append(train_auc_pr)
        history['val_loss'].append(avg_val_loss) # Validation Classification Loss
        history['val_accuracy'].append(val_acc)
        history['val_precision'].append(val_prec)
        history['val_recall'].append(val_recall)
        history['val_auc_roc'].append(val_auc_roc)
        history['val_auc_pr'].append(val_auc_pr)


        # print(f"Train Gen Loss: {avg_gen_loss:.4f}, Train Disc Loss: {avg_disc_loss:.4f}, Train Gen CLF Loss: {avg_gen_clf_loss:.4f}, Train Gen ADV Loss: {avg_gen_adv_loss:.4f}")
        # print(f"Train Acc: {train_acc:.4f}, Train Prec: {train_prec:.4f}, Train Recall: {train_recall:.4f}, Train AUC-ROC: {train_auc_roc:.4f}, Train AUC-PR: {train_auc_pr:.4f}")
        # print(f"Val Loss: {avg_val_loss:.4f}, Val Acc: {val_acc:.4f}, Val Prec: {val_prec:.4f}, Val Recall: {val_recall:.4f}, Val AUC-ROC: {val_auc_roc:.4f}, Val AUC-PR: {val_auc_pr:.4f}")


        # --- Early Stopping Check (using validation classification loss) ---
        if avg_val_loss < best_val_loss - early_stopping.min_delta:
            best_val_loss = avg_val_loss
            epochs_without_improvement = 0
            # Save current weights if validation loss improved
            generator.save_weights(generator_weights_path)
            discriminator.save_weights(discriminator_weights_path)
            #print("Saved generator and discriminator weights.")

        else:
            epochs_without_improvement += 1
            if epochs_without_improvement >= early_stopping.patience:
                print(f"Early stopping triggered based on validation loss after {epoch+1} epochs.")
                # Restore best weights if early stopping is triggered
                if os.path.exists(generator_weights_path):
                    print("Restoring generator weights from best validation epoch...")
                    generator.load_weights(generator_weights_path)
                if os.path.exists(discriminator_weights_path):
                    print("Restoring discriminator weights from best validation epoch...")
                    discriminator.load_weights(discriminator_weights_path)

                break # Exit the training loop

    return history # Return the history dictionary

In [ ]:
class PGFLossTF(tf.keras.losses.Loss):
    """
    Proposed Geometric Focal Loss (PGFL) for Imbalanced Datasets in TensorFlow.
    Combines dynamic class weighting based on confidence with Focal Loss.
    """
    def __init__(self, num_majority, num_minority, gamma=2.0, alpha_init=None, name="pgf_loss_tf"):
        super().__init__(name=name)
        self.gamma = tf.constant(gamma, dtype=tf.float32)

        # Initialize alpha (adaptive weight for minority class)
        # If alpha_init is not provided, use initial inverse frequency
        if alpha_init is None:
            initial_alpha = tf.cast(num_majority / (num_majority + num_minority), dtype=tf.float32)
        else:
            initial_alpha = tf.constant(alpha_init, dtype=tf.float32)

        # Make alpha a trainable variable for dynamic adaptation
        self.alpha = tf.Variable(initial_alpha, dtype=tf.float32, trainable=True, name="alpha")

        # Moving averages for confidence (non-trainable)
        self.mc0 = tf.Variable(0.0, dtype=tf.float32, trainable=False, name="mc0") # Confidence for correctly classified majority
        self.mc1 = tf.Variable(0.0, dtype=tf.float32, trainable=False, name="mc1") # Confidence for correctly classified minority
        self.momentum = tf.constant(0.99, dtype=tf.float32) # Momentum for moving averages

        # Track counts for updating moving averages (non-trainable)
        self.count0 = tf.Variable(0.0, dtype=tf.float32, trainable=False, name="count0")
        self.count1 = tf.Variable(0.0, dtype=tf.float32, trainable=False, name="count1")


    def update_confidence_moving_averages(self, y_true, y_pred_probs):
        """Updates the moving averages of confidence for correctly classified samples."""
        y_true = tf.cast(y_true, tf.float32)
        y_pred_probs = tf.cast(y_pred_probs, tf.float32)

        # Identify correctly classified samples
        predicted_labels = tf.cast(y_pred_probs > 0.5, tf.float32)
        correctly_classified_mask = tf.equal(predicted_labels, y_true)

        # Separate based on true class
        majority_mask = tf.equal(y_true, 0)
        minority_mask = tf.equal(y_true, 1)

        # Correctly classified majority samples
        cc_majority_mask = tf.logical_and(correctly_classified_mask, majority_mask)
        cc_majority_probs = tf.boolean_mask(1.0 - y_pred_probs, cc_majority_mask) # Confidence for majority is 1 - prob of minority
        num_cc_majority = tf.cast(tf.shape(cc_majority_probs)[0], tf.float32)

        # Correctly classified minority samples
        cc_minority_mask = tf.logical_and(correctly_classified_mask, minority_mask)
        cc_minority_probs = tf.boolean_mask(y_pred_probs, cc_minority_mask) # Confidence for minority is prob of minority
        num_cc_minority = tf.cast(tf.shape(cc_minority_probs)[0], tf.float32)


        # Update moving averages if there are correctly classified samples
        if num_cc_majority > 0:
            avg_cc_majority_confidence = tf.reduce_mean(cc_majority_probs)
            self.mc0.assign(self.momentum * self.mc0 + (1.0 - self.momentum) * avg_cc_majority_confidence)
            self.count0.assign(self.count0 + num_cc_majority)

        if num_cc_minority > 0:
            avg_cc_minority_confidence = tf.reduce_mean(cc_minority_probs)
            self.mc1.assign(self.momentum * self.mc1 + (1.0 - self.momentum) * avg_cc_minority_confidence)
            self.count1.assign(self.count1 + num_cc_minority)


    def update_alpha_dynamically(self):
        """Dynamically updates alpha based on the moving averages of confidence."""
        # Avoid division by zero if no samples of a class have been correctly classified yet
        epsilon = tf.keras.backend.epsilon()

        # Simple dynamic update: increase alpha if confidence in minority is lower than majority
        # Or, increase alpha if minority class is misclassified more often (indirectly via lower MC1)
        # This is a conceptual implementation, can be refined.
        # For example, alpha could be a sigmoid of a difference or ratio of confidences.

        # Let's try a simple ratio-based update for demonstration:
        # If MC1 is much lower than MC0, increase alpha towards 1.
        # If MC0 is much lower than MC1, decrease alpha towards 0.
        # Add epsilon to avoid division by zero
        confidence_ratio = (self.mc1 + epsilon) / (self.mc0 + epsilon)

        # Map the ratio to an alpha update. This mapping can be tuned.
        # A simple approach: if ratio < 1, increase alpha; if ratio > 1, decrease alpha.
        # The magnitude of change could depend on how far the ratio is from 1.
        # Example: alpha = sigmoid(k * (MC0 - MC1)) for some scaling factor k
        # Or, alpha = initial_alpha * (1 + delta * (MC0 - MC1))

        # Let's use a sigmoid based on the difference for a smoother update:
        # Scale the difference to make it more impactful if needed
        scaled_confidence_difference = 1.0 * (self.mc0 - self.mc1) # Adjust scaling factor as needed

        # Use sigmoid to map the difference to a value between 0 and 1
        # If MC0 > MC1 (majority easier/more confident), sigmoid(positive) > 0.5 -> increase alpha
        # If MC1 > MC0 (minority easier/more confident), sigmoid(negative) < 0.5 -> decrease alpha
        # This direction seems counter-intuitive based on the description "increase the weight of the class that the model is less confident about".
        # Let's adjust: increase alpha if MC1 (minority confidence) is low relative to MC0 (majority confidence).
        # So, we want alpha to increase as (MC0 - MC1) increases. Sigmoid(MC0 - MC1) works for this.

        # Let's try a simpler update rule first: adjust alpha based on the sign of the confidence difference.
        # If minority confidence is lower than majority confidence (MC1 < MC0), slightly increase alpha.
        # If majority confidence is lower than minority confidence (MC0 < MC1), slightly decrease alpha.
        # This requires a learning rate for alpha itself.

        # A better approach might be to directly train alpha as a variable, as initially intended.
        # The moving averages MC0 and MC1 can then be used *within* the loss function's calculation
        # to modulate the per-sample loss based on confidence, rather than directly updating alpha.

        # Let's stick to the original plan of training alpha and use MC0/MC1 for monitoring or
        # potentially in a more complex update rule if needed later.
        # For now, the trainable self.alpha is the 'adaptive' part. The moving averages MC0 and MC1
        # can serve as indicators of which class the model is struggling with, which might guide
        # hyperparameter tuning of the initial alpha or the learning rate for alpha.

        # Let's keep self.alpha trainable and remove the direct update based on MC0/MC1 for now.
        # The 'adaptiveness' comes from the optimizer adjusting self.alpha during training.
        pass # No direct update to self.alpha here

    def call(self, y_true, y_pred_logits):
        y_true = tf.cast(y_true, tf.float32)
        y_pred_logits = tf.cast(y_pred_logits, tf.float32)

        # Calculate probabilities
        p = tf.sigmoid(y_pred_logits)

        # Update moving averages of confidence with the current batch's predictions
        # This should ideally be done after the optimizer step, but for simplicity in a single call method,
        # we'll update it here. This makes the update slightly delayed.
        # A more accurate approach might involve a custom training loop that updates MC0/MC1 after gradients are applied.
        # For now, let's update it here for demonstration.
        self.update_confidence_moving_averages(y_true, p)


        # Calculate standard BCE
        bce_unweighted = tf.nn.sigmoid_cross_entropy_with_logits(labels=y_true, logits=y_pred_logits)

        # p_t is the probability of the true class
        p_t = y_true * p + (1 - y_true) * (1 - p)

        # Modulating factor (1 - p_t)^gamma
        modulating_factor = tf.pow(1.0 - p_t, self.gamma)

        # Adaptive alpha_t: Use the trainable self.alpha
        # alpha_t is alpha for positive (minority), 1-alpha for negative (majority)
        alpha_t = y_true * self.alpha + (1 - y_true) * (1.0 - self.alpha)

        # PGFL Loss: alpha_t * (1 - p_t)^gamma * BCE
        pgf_loss = alpha_t * modulating_factor * bce_unweighted

        # Add print statements for debugging
        # tf.print("y_true:", y_true, summarize=-1)
        # tf.print("y_pred_logits:", y_pred_logits, summarize=-1)
        # tf.print("p:", p, summarize=-1)
        # tf.print("bce_unweighted:", bce_unweighted, summarize=-1)
        # tf.print("p_t:", p_t, summarize=-1)
        # tf.print("modulating_factor:", modulating_factor, summarize=-1)
        # tf.print("alpha_t:", alpha_t, summarize=-1)
        # tf.print("pgf_loss (per sample):", pgf_loss, summarize=-1)
        # tf.print("pgf_loss (mean):", tf.reduce_mean(pgf_loss), summarize=-1)

        return tf.reduce_mean(pgf_loss)

    @property
    def trainable_variables(self):
        # The only trainable variable in this loss is alpha
        return [self.alpha]

    def reset_metrics(self):
        """Resets the state of the moving average variables."""
        self.mc0.assign(0.0)
        self.mc1.assign(0.0)
        self.count0.assign(0.0)
        self.count1.assign(0.0)
        # Note: self.alpha is trainable and should not be reset between epochs/files
        # unless specifically desired. For comparison across files, the model and loss
        # are re-initialized, which handles this.

In [ ]:
  # Custom train step for PGFL with gradient clipping
# Removed @tf.function decorator
def train_step_pgfl(model, loss_function, optimizer, X_batch, y_batch, clip_norm):
    with tf.GradientTape() as tape:
        y_pred_logits = model(X_batch, training=True) # Set training=True for dropout
        loss = loss_function(y_batch, y_pred_logits) # loss.call calculates loss

    # Update moving averages *after* calculating the loss and outside the gradient tape
    # This ensures the update uses the gradients from the loss calculation
    y_pred_probs = tf.sigmoid(y_pred_logits)
    if isinstance(loss_function, PGFLossTF): # Only update for PGFLossTF
        loss_function.update_confidence_moving_averages(y_batch, y_pred_probs)


    # Get gradients for model's trainable variables AND loss function's trainable variable (alpha)
    trainable_variables = model.trainable_variables + loss_function.trainable_variables if hasattr(loss_function, 'trainable_variables') else model.trainable_variables
    gradients = tape.gradient(loss, trainable_variables)

    # Add a check for None gradients and filter
    if any(grad is None for grad in gradients):
        none_grads = [var.name for grad, var in zip(gradients, trainable_variables) if grad is None]
        # print(f"Warning: Gradients are None for variables: {none_grads}") # Keep this for debugging if needed
        grads_and_vars = [(grad, var) for grad, var in zip(gradients, trainable_variables) if grad is not None]
        if not grads_and_vars:
            raise ValueError("All gradients are None. Cannot apply gradients.")
    else:
        grads_and_vars = list(zip(gradients, trainable_variables))


    # Apply gradient clipping
    clipped_gradients, _ = tf.clip_by_global_norm([g for g, v in grads_and_vars], clip_norm)

    # Apply gradients
    optimizer.apply_gradients(zip(clipped_gradients, [v for g, v in grads_and_vars]))


    return loss

# Custom test step for PGFL (used for validation)
# Removed @tf.function decorator
def test_step_pgfl(model, loss_function, X_batch, y_batch, metrics):
    y_pred_logits = model(X_batch, training=False) # Set training=False for dropout
    loss = loss_function(y_batch, y_pred_logits) # loss.call updates MC0/MC1 (can stay in call for test, or move out)

    # Update metrics
    y_pred_probs = tf.sigmoid(y_pred_logits)
    metrics['accuracy'].update_state(y_batch, y_pred_probs)
    metrics['precision'].update_state(y_batch, y_pred_probs)
    metrics['recall'].update_state(y_batch, y_pred_probs)
    metrics['auc_roc'].update_state(y_batch, y_pred_probs)
    metrics['auc_pr'].update_state(y_batch, y_pred_probs)

    return loss

# Custom training loop for PGFL
def custom_train_loop_pgfl(model, loss_function, optimizer, train_dataset, val_dataset, epochs, early_stopping, clip_norm):
    best_val_loss = float('inf')
    epochs_without_improvement = 0
    # Define paths for saving temporary weights for both models
    model_weights_path = 'temp_model_weights_pgfl.weights.h5' # Separate paths for PGFL
    loss_weights_path = 'temp_loss_weights_pgfl.npy' # Separate paths for PGFL

    history = {
        'loss': [], 'accuracy': [], 'precision': [], 'recall': [], 'auc_roc': [], 'auc_pr': [],
        'val_loss': [], 'val_accuracy': [], 'val_precision': [], 'val_recall': [], 'val_auc_roc': [], 'val_auc_pr': []
    }

    train_metrics = {
        'accuracy': BinaryAccuracy(threshold=0.5), 'precision': Precision(thresholds=0.5),
        'recall': Recall(thresholds=0.5), 'auc_roc': AUC(), 'auc_pr': AUC(curve='PR')
    }
    val_metrics = {
        'accuracy': BinaryAccuracy(threshold=0.5), 'precision': Precision(thresholds=0.5),
        'recall': Recall(thresholds=0.5), 'auc_roc': AUC(), 'auc_pr': AUC(curve='PR')
    }


    for epoch in range(epochs):
        print(f"\nEpoch {epoch+1}/{epochs}")

        for metric in train_metrics.values(): metric.reset_state()
        for metric in val_metrics.values(): metric.reset_state()
        # Reset loss function's internal metrics (MC0, MC1) at the start of each epoch?
        # Or let them accumulate? The description suggests moving averages over time,
        # so perhaps don't reset per epoch, but per file. The re-initialization per file handles this.
        # loss_function.reset_metrics() # Only if resetting per epoch is desired


        total_train_loss = 0
        num_train_batches = 0
        for i, (X_batch, y_batch) in enumerate(train_dataset): # Add enumerate to get batch index
            loss = train_step_pgfl(model, loss_function, optimizer, X_batch, y_batch, clip_norm)
            total_train_loss += loss.numpy()
            num_train_batches += 1
            # print(f"  Batch {i}: Loss = {loss.numpy():.4f}") # Print loss per batch


            # Update train metrics manually
            y_pred_logits = model(X_batch, training=False)
            y_pred_probs = tf.sigmoid(y_pred_logits)
            train_metrics['accuracy'].update_state(y_batch, y_pred_probs)
            train_metrics['precision'].update_state(y_batch, y_pred_probs)
            train_metrics['recall'].update_state(y_batch, y_pred_probs)
            train_metrics['auc_roc'].update_state(y_batch, y_pred_probs)
            train_metrics['auc_pr'].update_state(y_batch, y_pred_probs)


        avg_train_loss = total_train_loss / num_train_batches

        total_val_loss = 0
        num_val_batches = 0
        for X_batch_val, y_batch_val in val_dataset:
            loss = test_step_pgfl(model, loss_function, X_batch_val, y_batch_val, val_metrics)
            total_val_loss += loss.numpy()
            num_val_batches += 1

        avg_val_loss = total_val_loss / num_val_batches

        train_acc = train_metrics['accuracy'].result().numpy()
        train_prec = train_metrics['precision'].result().numpy()
        train_recall = train_metrics['recall'].result().numpy()
        train_auc_roc = train_metrics['auc_roc'].result().numpy()
        train_auc_pr = train_metrics['auc_pr'].result().numpy()

        val_acc = val_metrics['accuracy'].result().numpy()
        val_prec = val_metrics['precision'].result().numpy()
        val_recall = val_metrics['recall'].result().numpy()
        val_auc_roc = val_metrics['auc_roc'].result().numpy()
        val_auc_pr = val_metrics['auc_pr'].result().numpy()

        history['loss'].append(avg_train_loss)
        history['accuracy'].append(train_acc)
        history['precision'].append(train_prec)
        history['recall'].append(train_recall)
        history['auc_roc'].append(train_auc_roc)
        history['auc_pr'].append(train_auc_pr)
        history['val_loss'].append(avg_val_loss)
        history['val_accuracy'].append(val_acc)
        history['val_precision'].append(val_prec)
        history['val_recall'].append(val_recall)
        history['val_auc_roc'].append(val_auc_roc)
        history['val_auc_pr'].append(val_auc_pr)


        print(f"Train Loss: {avg_train_loss:.4f}, Train Acc: {train_acc:.4f}, Train Prec: {train_prec:.4f}, Train Recall: {train_recall:.4f}, Train AUC-ROC: {train_auc_roc:.4f}, Train AUC-PR: {train_auc_pr:.4f}")
        print(f"Val Loss: {avg_val_loss:.4f}, Val Acc: {val_acc:.4f}, Val Prec: {val_prec:.4f}, Val Recall: {val_recall:.4f}, Val AUC-ROC: {val_auc_roc:.4f}, Val AUC-PR: {val_auc_pr:.4f}")

        # Print the current value of the trainable alpha and moving averages
        if isinstance(loss_function, PGFLossTF):
             print(f"  PGFL Learned Alpha: {loss_function.alpha.numpy():.4f}, MC0: {loss_function.mc0.numpy():.4f}, MC1: {loss_function.mc1.numpy():.4f}")


        # --- Early Stopping Check (using validation loss) ---
        if avg_val_loss < best_val_loss - early_stopping.min_delta:
            best_val_loss = avg_val_loss
            epochs_without_improvement = 0
            # Save current weights if validation loss improved
            model.save_weights(model_weights_path)
            # Save loss function trainable variables (alpha) and moving averages
            np.save(loss_weights_path, [loss_function.alpha.numpy(), loss_function.mc0.numpy(), loss_function.mc1.numpy()])
            #print("Saved model and PGFL loss parameters.")

        else:
            epochs_without_improvement += 1
            if epochs_without_improvement >= early_stopping.patience:
                print(f"Early stopping triggered based on validation loss after {epoch+1} epochs.")
                # Restore best weights if early stopping is triggered
                if os.path.exists(model_weights_path):
                    print("Restoring model weights from best validation epoch...")
                    model.load_weights(model_weights_path)
                if os.path.exists(loss_weights_path):
                    print("Restoring PGFL loss parameters from best validation epoch...")
                    best_loss_params = np.load(loss_weights_path)
                    loss_function.alpha.assign(best_loss_params[0])
                    loss_function.mc0.assign(best_loss_params[1])
                    loss_function.mc1.assign(best_loss_params[2])
                    print(f"  Restored PGFL Params: Alpha={loss_function.alpha.numpy():.4f}, MC0={loss_function.mc0.numpy():.4f}, MC1={loss_function.mc1.numpy():.4f}")

                break # Exit the training loop

    return history # Return the history dictionary


In [ ]:



# --- 2. Data Loading from Google Drive ---
# This function is now the sole method for loading data.
def load_data_from_drive(file_path):
    """
    Loads a dataset from a given file path.
    Assumes the last column is the target and no header row in the CSV file.
    Args:
        file_path (str): The full path to the CSV file.
    Returns:
        pd.DataFrame: Loaded DataFrame.
    """
    if not os.path.exists(file_path):
        print(f"Error: File not found at {file_path}.")
        return None

    # Read CSV assuming with header. P
    # Assuming the files are Excel (.xlsx) as per the main execution block
    try:
        df = pd.read_excel(file_path)
        # doing it for non avalanche datasets otherwise avalanceh datasets have headers
        #df = pd.read_excel(file_path, header=None)

        # Drop columns by name if they exist
        df = df.drop(columns=['Date', 'Index'], errors='ignore')

        print(f"DataFrame after dropping 'Date' and 'Index' columns: {df.shape}")
        print(f"Remaining columns: {df.columns.tolist()}")

        # Show class distribution of the assumed target (last column)
        print(f"Assuming the last column '{df.columns[-1]}' is the target column.")
        print(f"Class distribution:\n{df.iloc[:, -1].value_counts()}")

        return df
    except Exception as e:
        print(f"Error loading file {file_path}: {e}")
        return None


# --- 4. Loss Function Implementations for TensorFlow ---























Mounted at /content/drive


**Reasoning**:
Implement early stopping logic within the custom training loop to stop training when validation loss does not improve for a specified number of epochs.



**Reasoning**:
The `RuntimeError: A non-persistent GradientTape can only be used to compute one set of gradients (or jacobians)` indicates that `tape.gradient()` is being called multiple times on the same non-persistent tape within the `train_step` function. This happens because I attempted to calculate gradients for `model.trainable_variables` and `loss_trainable_variables` separately within the same `with tf.GradientTape() as tape:` block.

To fix this, I need to collect all variables for which gradients should be computed *before* calling `tape.gradient()`. Then, I will call `tape.gradient()` only once, passing the loss and the combined list of model and loss trainable variables.

This approach ensures that the gradients are computed in a single pass over the tape, resolving the `RuntimeError` and allowing the training loop to proceed. The early stopping logic remains correctly implemented in the `custom_train_loop`.



In [ ]:
class ClassBalancedLossTF(tf.keras.losses.Loss):
    """
    Class Balanced Loss for Imbalanced Datasets in TensorFlow.
    Re-weights the standard loss by the 'effective number of samples' for each class.
    """
    def __init__(self, num_majority, num_minority, beta=0.999, name="class_balanced_loss_tf"):
        super().__init__(name=name)
        self.beta = tf.constant(beta, dtype=tf.float32)

        # Calculate effective number of samples for each class
        # Add a small epsilon to avoid division by zero in case num_samples is 0 or beta is 1
        epsilon = tf.keras.backend.epsilon()

        effective_num_majority = (1.0 - tf.pow(self.beta, tf.cast(num_majority, tf.float32))) / (1.0 - self.beta + epsilon)
        effective_num_minority = (1.0 - tf.pow(self.beta, tf.cast(num_minority, tf.float32))) / (1.0 - self.beta + epsilon)

        # Calculate class-balanced weights
        self.w0_cb = tf.constant(1.0, dtype=tf.float32) / effective_num_majority
        self.w1_cb = tf.constant(1.0, dtype=tf.float32) / effective_num_minority

        tf.print(f"CB Loss Weights: w0 (majority)={self.w0_cb}, w1 (minority)={self.w1_cb}")

    def call(self, y_true, y_pred_logits):
        y_true = tf.cast(y_true, tf.float32)
        y_pred_logits = tf.cast(y_pred_logits, tf.float32)

        # Calculate standard BCE
        bce_unweighted = tf.nn.sigmoid_cross_entropy_with_logits(labels=y_true, logits=y_pred_logits)

        # Apply class-balanced weights
        cb_loss = tf.where(tf.equal(y_true, 1), bce_unweighted * self.w1_cb, bce_unweighted * self.w0_cb)

        return tf.reduce_mean(cb_loss)

In [ ]:
class FocalLossTF(tf.keras.losses.Loss):
    """
    Focal Loss for Imbalanced Datasets in TensorFlow.
    Focuses training on hard, misclassified examples by down-weighting easy examples.
    """
    def __init__(self, alpha=0.25, gamma=2.0, name="focal_loss_tf"):
        super().__init__(name=name)
        self.alpha = tf.constant(alpha, dtype=tf.float32)
        self.gamma = tf.constant(gamma, dtype=tf.float32)

    def call(self, y_true, y_pred_logits):
        y_true = tf.cast(y_true, tf.float32)
        y_pred_logits = tf.cast(y_pred_logits, tf.float32)

        # Calculate standard BCE (log loss)
        bce = tf.nn.sigmoid_cross_entropy_with_logits(labels=y_true, logits=y_pred_logits)

        # Calculate probabilities
        p = tf.sigmoid(y_pred_logits)

        # p_t is the probability of the true class
        p_t = y_true * p + (1 - y_true) * (1 - p)

        # alpha_t is alpha for positive, 1-alpha for negative
        alpha_t = y_true * self.alpha + (1 - y_true) * (1.0 - self.alpha)

        # Modulating factor (1 - p_t)^gamma
        modulating_factor = tf.pow(1.0 - p_t, self.gamma)

        # Focal Loss: alpha_t * (1 - p_t)^gamma * BCE
        focal_loss = alpha_t * modulating_factor * bce
        return tf.reduce_mean(focal_loss)

In [ ]:
# Define the plots save directory again as it's needed for the print statement
plots_save_dir = os.path.join(GOOGLE_DRIVE_PATH, 'training_plots')

# print("Manually reviewing the training history plots saved in Google Drive.")
# print(f"Looking for plots named like '*AMRL_*_plot.png' in {plots_save_dir} for the AMRL loss.")
# print("Specifically checking the plots for:")
# print("1. Training and Validation Loss: How did they change over epochs? Did they converge? Was there overfitting?")
# print("2. Training and Validation Accuracy, Precision, Recall, AUC-ROC, AUC-PR: What were the trends for these metrics?")
# print("3. Note the epoch where early stopping occurred (indicated by the training output) and the best validation loss.")
# print("4. Examine the learned AMRL parameters printed per epoch during the custom training loop to see their evolution.")

# Note: Accessing external files (like plots in Google Drive) for analysis programmatically
# in this environment without explicit file reading/display code is not straightforward.
# The analysis step relies on manual inspection of the generated files and the printed output.

## Define discriminator model

### Subtask:
Create a new TensorFlow/Keras model for the discriminator network. This network will take the model's output (logits or probabilities) and potentially the input features as input and try to distinguish between real (true label) and fake (predicted label) samples.


## Define gan loss function

### Subtask:
Implement the adversarial loss function. This will likely involve combining the standard classification loss (like BCE or one of the imbalanced losses) with the discriminator's loss. The generator (your current model) will try to minimize the loss from the discriminator, while the discriminator will try to maximize it.


**Reasoning**:
Define the PGFL loss function class as requested in the instructions.



**Reasoning**:
Implement the GAN-based loss function components within the custom training loop structure, as defining a single `tf.keras.losses.Loss` class for GANs with separate generator and discriminator steps is not standard.



**Reasoning**:
Integrate the GAN training loop into the main execution block to train the model using the GAN-based approach for comparison with other loss functions.



In [ ]:
# Modify the main execution block to include 'GAN' in the comparison

# Define a function for model initialization
def initialize_model(input_dim, loss_type, num_majority_train, num_minority_train):
    model = SimpleMLP_TF(input_dim)
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001) # Optimizer for the main model (Generator)

    discriminator_model = None # Discriminator is only needed for GAN loss
    discriminator_optimizer = None # Discriminator optimizer is only needed for GAN loss

    minority_loss_weight = None # Trainable weight for Meta-Learning
    meta_optimizer = None # Optimizer for Meta-Learning weight

    loss_function = None # Classification loss function (used directly or as base for GAN/Meta-Learning)

    if loss_type == 'AMRL':
        w1_bce_amrl = num_majority_train / (num_minority_train + num_majority_train)
        w0_bce_amrl = num_minority_train / (num_minority_train + num_majority_train)
        loss_function = AMRLLossTF(w1 = w1_bce_amrl, w0 = w0_bce_amrl, m1_init = M_POS_INIT, m0_init = M_NEG_INIT,
                                  lambda1_init=LAMBDA_POS_INIT, lambda0_init = LAMBDA_NEG_INIT,
                                  beta1 = BETA_POS_INIT, beta0 = BETA_NEG_INIT)
    elif loss_type == 'PGFL':
        alpha_init_pgfl = num_majority_train / (num_majority_train + num_minority_train)
        loss_function = PGFLossTF(num_majority=num_majority_train, num_minority=num_minority_train,
                                  gamma=gamma_pgfl, alpha_init=alpha_init_pgfl)
    elif loss_type == 'GAN':
        classification_loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True)
        # Fix: Directly use input_dim to initialize Discriminator
        discriminator_model = Discriminator(input_dim_X=input_dim, input_dim_Y=1)
        discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001) # Separate optimizer for discriminator
        loss_function = classification_loss_fn # Use base classification loss for GAN generator metrics tracking

    elif loss_type in ['BCE', 'FOCAL', 'CB']:
        metrics_list = [
            BinaryAccuracy(name = 'accuracy', threshold = 0.5),
            Precision(name = 'precision', thresholds = 0.5),
            Recall(name = 'recall', thresholds = 0.5),
            AUC(name = 'auc_roc'),
            AUC(curve = 'PR', name ='auc_pr')
        ]
        if loss_type == 'BCE':
            loss_function = tf.keras.losses.BinaryCrossentropy(from_logits = True)
        elif loss_type == 'FOCAL':
            alpha_focal = num_majority_train / (num_minority_train + num_minority_train)
            gamma_focal = 2.0
            loss_function = FocalLossTF(alpha = alpha_focal, gamma = gamma_focal)
        elif loss_type == 'CB':
            beta_cb = 0.999
            loss_function = ClassBalancedLossTF(num_majority = num_majority_train, num_minority = num_minority_train, beta = beta_cb)

        # Compile the model for Keras's fit method
        model.compile(optimizer=optimizer, loss=loss_function, metrics=metrics_list)


    return model, optimizer, loss_function, discriminator_model, discriminator_optimizer


# Define a function for training the model
def train_model(model, optimizer, loss_function, discriminator_model, discriminator_optimizer,
                train_dataset, val_dataset, epochs, early_stopping_callback, clip_norm, loss_type):
    history = None
    if loss_type == 'AMRL':
        print("\nStarting custom training loop for AMRL...")
        history = custom_train_loop(model, loss_function, optimizer, train_dataset, val_dataset, epochs, early_stopping_callback, clip_norm)
        print("Custom training loop finished.")
    elif loss_type == 'PGFL':
        print("\nStarting custom training loop for PGFL...")
        history = custom_train_loop_pgfl(model, loss_function, optimizer, train_dataset, val_dataset, epochs, early_stopping_callback, CLIP_NORM)
        print("Custom training loop finished.")
    elif loss_type == 'GAN':
         print("\nStarting custom training loop for GAN...")
         # For GAN, the base classification loss is typically BCE for the generator
         classification_loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True)
         history = custom_train_loop_gan(model, discriminator_model, optimizer, discriminator_optimizer,
                                                classification_loss_fn, train_dataset, val_dataset,
                                                epochs, early_stopping_callback, CLIP_NORM, GAN_LAMBDA)
         print("Custom training loop finished.")
    elif loss_type in ['BCE', 'FOCAL', 'CB']:
        print("\nStarting model.fit training...")
        # The model is already compiled in initialize_model for these loss types
        history = model.fit(train_dataset.unbatch().map(lambda x, y: (x, y)).batch(BATCH_SIZE), # Re-batch for model.fit
                            epochs=epochs,
                            validation_data=val_dataset.unbatch().map(lambda x, y: (x, y)).batch(BATCH_SIZE), # Re-batch for model.fit
                            callbacks=[early_stopping_callback],
                            verbose=1)
        history = history.history
        print("model.fit training finished.")

    return history


# Define a function for evaluating the model and collecting results
def evaluate_model(model, loss_type, history, X_test_scaled, y_test_tf, y_test, BATCH_SIZE, GAN_LAMBDA):
    print("\nEvaluating on the Test Set...")

    # Evaluate the model using model.evaluate or manual calculation for custom loop
    # Calculate metrics manually using predictions for all loss types for consistency
    y_pred_logits_test = model.predict(X_test_scaled, batch_size=BATCH_SIZE)
    y_pred_probs_test = tf.sigmoid(y_pred_logits_test).numpy().flatten()
    y_pred_binary_test = (y_pred_probs_test > 0.5).astype(int)
    y_true_test_flat = y_test.values.flatten()

    # 1. Get predicted probabilities from model
    y_pred_probs = model.predict(X_test_scaled, batch_size=BATCH_SIZE)

    # 2. Convert probabilities to class predictions (threshold = 0.5)
    y_pred_labels = (y_pred_probs >= 0.5).astype(int)

    # 3. Ensure y_test is in correct format (binary integers)
    y_true = np.array(y_test_tf).astype(int)

    # 4. Calculate metrics
    accuracy = round(accuracy_score(y_true, y_pred_labels),2)
    precision = round(precision_score(y_true, y_pred_labels, zero_division=0),2)
    recall = round(recall_score(y_true, y_pred_labels, zero_division=0),2)
    auc_roc = round(roc_auc_score(y_true, y_pred_probs),2)
    auc_pr = round(average_precision_score(y_true, y_pred_probs),2)

    # Confusion Matrix components
    tn, fp, fn, tp = confusion_matrix(y_true_test_flat, y_pred_binary_test, labels=[0, 1]).ravel()

    # Specificity (True Negative Rate)
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0.0

    # Geometric Mean (G-mean)
    current_recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    specificity_gmean = tn / (tn + fp) if (tn + fp) > 0 else 0.0
    g_mean = np.sqrt(current_recall * specificity_gmean) if current_recall >= 0 and specificity_gmean >= 0 else np.nan
    g_mean = round(g_mean, 2)


    # Balanced Accuracy
    bal_accuracy = balanced_accuracy_score(y_true_test_flat, y_pred_binary_test)
    bal_accuracy = round(bal_accuracy, 2)

    # Kuipers Skill Score (KSS) / True Skill Score (TSS)
    kss_tss = current_recall + specificity - 1
    kss_tss = round(kss_tss, 2)

    # Heidke Skill Score (HSS)
    far = false_alarm(tp,tn,fp,fn)
    bias = bias_score(tp,tn,fp,fn)
    pod = probability_of_detection(tp,tn,fp,fn)
    hss= heidke_skill_score(tp,tn,fp,fn)
    pss = pierce_skill_score(tp,tn,fp,fn)
    f1  =  f1_score(tp,tn,fp,fn)

    num_minority_test = np.sum(y_test)
    num_majority_test = len(y_test) - num_minority_test # Corrected from num_minority_test

    #Combine Keras metrics and manually calculated metrics
    test_metrics_dict = {
        'Dataset': os.path.basename(current_file_name), # Use the current file name
        'Loss_Type': loss_type,
        #'Loss': loss, # Loss is not directly comparable across loss functions on the test set
        #'Epochs': len(history['loss']) if loss_type not in ['GAN', 'METALEARNING'] else (len(history['gen_loss']) if loss_type == 'GAN' else len(history['meta_val_loss'])), # Get epochs from appropriate history
        'Epochs': len(history['loss']) if loss_type not in ['GAN'] else (len(history['gen_loss']) if loss_type == 'GAN' else len(history['meta_val_loss'])), # Get epochs from appropriate history
        'Batch_Size': BATCH_SIZE,
        'Positive': num_minority_test,
        'Negative': num_majority_test,
        'TP': tp,
        'TN': tn,
        'FP': fp,
        'FN': fn,
        'Accuracy' : accuracy,
        'Precision': precision,
        'Recall': recall,
        'AUC-ROC': auc_roc,
        'AUC-PR': auc_pr,
        'Specificity': specificity,
        'G-Mean': g_mean,
        'Balanced_Accuracy': bal_accuracy,
        'KSS_TSS': kss_tss,
        'HSS': hss,
        'PSS': pss,
        'FAR': far,
        'bias': bias,
        'pod': pod,
        'f1': f1
    }


    # If the loss type is AMRL, add the learned parameters to the results dictionary
    if loss_type == 'AMRL':
         # Need to access the loss function object from the training loop
         # This requires passing it or retrieving it if it's a global/accessible variable
         # For now, assuming loss_function is still available from the training scope
         if 'loss_function' in locals() and isinstance(loss_function, AMRLLossTF):
            test_metrics_dict['Learned_M_POS'] = loss_function.m1.numpy()
            test_metrics_dict['Learned_M_NEG'] = loss_function.m0.numpy()
            test_metrics_dict['Learned_LAMBDA_POS'] = loss_function.lambda1.numpy()
            test_metrics_dict['Learned_LAMBDA_NEG'] = loss_function.lambda0.numpy()

    # If the loss type is PGFL, add the learned alpha and final moving averages
    elif loss_type == 'PGFL':
         if 'loss_function' in locals() and isinstance(loss_function, PGFLossTF):
            test_metrics_dict['Learned_Alpha'] = loss_function.alpha.numpy()
            test_metrics_dict['Final_MC0'] = loss_function.mc0.numpy()
            test_metrics_dict['Final_MC1'] = loss_function.mc1.numpy()
    # If the loss type is GAN, add the GAN_LAMBDA parameter
    elif loss_type == 'GAN':
         test_metrics_dict['GAN_Lambda'] = GAN_LAMBDA
    # If the loss type is METALEARNING, add the final learned minority weight
    # elif loss_type == 'METALEARNING':
    #      if minority_loss_weight is not None: # Check if the weight was initialized
    #         test_metrics_dict['Final_Minority_Weight'] = minority_loss_weight.numpy()


    for metric, value in test_metrics_dict.items():
        if isinstance(value, float):
            print(f"{metric}: {value:.4f}")
        else:
            print(f"{metric}: {value}")

    return test_metrics_dict, history # Return history as well for plotting


if __name__ == "__main__":
    # Add 'GAN' to the list of loss functions to compare
    LOSS_TYPES_TO_COMPARE = ['AMRL', 'BCE', 'FOCAL', 'CB', 'PGFL', 'GAN']
    # LOSS_TYPES_TO_COMPARE = ['METALEARNING', 'BCE'] # Add METALEARNING to the list

    # --- Configuration for data loading ---
    # Ensure GOOGLE_DRIVE_PATH is set and exists
    if 'GOOGLE_DRIVE_PATH' not in globals() or not os.path.exists(GOOGLE_DRIVE_PATH):
        # This check should now pass after the previous cell
        raise ValueError("GOOGLE_DRIVE_PATH not set or does not exist. Please configure it to your Google Drive folder containing XLSX files.")

    print(f"Scanning for XLSX files in: {GOOGLE_DRIVE_PATH}")

    file_paths_to_process = [os.path.join(GOOGLE_DRIVE_PATH, f) for f in os.listdir(GOOGLE_DRIVE_PATH) if f.endswith('.xlsx')]
    # sort the files
    file_paths_to_process = sorted(file_paths_to_process, key=lambda x: os.path.basename(x).lower())

    if not file_paths_to_process:
        print(f"No XLSX files found in {GOOGLE_DRIVE_PATH}. Please check the path and file extensions.")
        # exit() # Don't exit in a single cell, just print message and continue

    # Create directories to save plots and text files within the Google Drive path
    plots_save_dir = os.path.join(GOOGLE_DRIVE_PATH, 'training_plots')
    os.makedirs(plots_save_dir, exist_ok=True)
    print(f"Plots will be saved to: {plots_save_dir}")

    results_save_dir = os.path.join(GOOGLE_DRIVE_PATH, 'text files')
    os.makedirs(results_save_dir, exist_ok=True)
    print(f"Results will be saved to: {results_save_dir}")


    for file_path in file_paths_to_process:
        current_file_name = os.path.basename(file_path)
        print(f"\n############################################################")
        print(f"Processing Dataset: {current_file_name}")
        print(f"############################################################")

        # Load data for the current file
        df = load_data_from_drive(file_path)
        if df is None:
            print(f"Skipping {current_file_name} due to loading error.")
            continue # Skip to the next file

        # Define features (X) and target (y)
        # Assumes the last column is always the target

        # Remove certain columns whihc i think are not relevant
        print(df.head())
        # FIX: Drop 'Date' column explicitly from features before splitting
        df_new = df.drop(['Date', 'Station_Code', 'Cloud_Amount', 'Sunshine', 'Fresh_Snow_WEq', 'Snow_Penetration', 'Snow_Temperature'], axis=1, errors='ignore')

        # take only the rows upto 1351 and ignore records from 01 Nov 2023 onwards since occurences very lss after that
        df_new = df_new.iloc[:4351, :]
        # Drop rows with any NaN values
        df_new = df_new.dropna()

        # Ensure the target is the last column after dropping
        X = df_new.iloc[:, :-1]
        y = df_new.iloc[:, -1]


        # --- Data Preprocessing and Splitting (for current dataset) ---
        print("\nSplitting data into train, validation, and test sets...")
        # Use the processed X and y for splitting
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42, stratify = y)
        X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = (0.1/0.8), random_state=42, stratify = y_train)

        print(f"Train set size: {len(X_train)} (Minority: {y_train.sum()})")
        print(f"Validation set size: {len(X_val)} (Minority: {y_val.sum()})")
        print(f"Test set size: {len(X_test)} (Minority: {y_test.sum()})") # Corrected to y_test.sum()


        # Collect ratios
        ratios_results = []
        for name, labels in [("Train", y_train), ("Validation", y_val), ("Test", y_test)]:
          counts, ratio, imbalance_ratio = get_class_ratio(labels.values.flatten()) # Ensure labels are flattened
          ratio_str = ", ".join([f"{cls}: {ratio[cls]:.{2}f}" for cls in ratio])
          ratios_results.append(
            f"{name} set:\n"
            f" Counts: {counts.tolist()} (0s={counts[0]}, 1s={counts[1]})\n" # Access counts correctly
            f" Ratios: {{{ratio_str}}}\n"
            f" Imbalance Ratio (maj:min): {imbalance_ratio}\n"
            )

        # Save to text file
        ratios_file_path = os.path.join(results_save_dir, f"{current_file_name}_class_ratios.txt")
        print(f"Saving class ratios to: {ratios_file_path}")
        with open(ratios_file_path, "w") as f:
          for res in ratios_results:
            f.write(res + "\n")



        scaler = StandardScaler()
        # Apply scaler to the numerical features
        X_train_scaled = scaler.fit_transform(X_train)
        X_val_scaled = scaler.transform(X_val)
        X_test_scaled = scaler.transform(X_test)

        y_train_tf = y_train.values.astype(np.float32).reshape(-1, 1)
        y_val_tf = y_val.values.astype(np.float32).reshape(-1, 1)
        y_test_tf = y_test.values.astype(np.float32).reshape(-1, 1)


        # Create TensorFlow Datasets for custom training loop
        train_dataset_tf = tf.data.Dataset.from_tensor_slices((X_train_scaled.astype(np.float32), y_train_tf)).batch(BATCH_SIZE).shuffle(buffer_size=len(X_train_scaled))
        # Split the validation set further for meta-learning (even if not using meta-learning in this run, keep for consistency)
        X_val_inner, X_meta_val, y_val_inner, y_meta_val = train_test_split(
            X_val_scaled, y_val_tf, test_size=0.5, random_state=42, stratify=y_val_tf
        )
        val_dataset_inner = tf.data.Dataset.from_tensor_slices((X_val_inner.astype(np.float32), y_val_inner)).batch(BATCH_SIZE)
        meta_val_dataset_tf = tf.data.Dataset.from_tensor_slices((X_meta_val.astype(np.float32), y_meta_val)).batch(BATCH_SIZE)

        # test_dataset_tf = tf.data.Dataset.from_tensor_slices((X_test_scaled.astype(np.float32), y_test_tf)).batch(BATCH_SIZE)

        # --- Loss Function Parameters (common calculations for current dataset) ---
        num_minority_train = np.sum(y_train)
        num_majority_train = len(y_train) - num_minority_train


        w1_bce_amrl = num_majority_train / (num_minority_train + num_majority_train)
        w0_bce_amrl = num_minority_train / (num_minority_train + num_majority_train)

        # Parameters for Focal Loss
        alpha_focal = num_majority_train / (num_majority_train + num_minority_train) # Corrected denominator
        gamma_focal = 2.0

        # Parameters for Class Balanced Loss
        beta_cb = 0.999

        # Parameters for PGFL
        gamma_pgfl = 2.0
        # Initialize alpha for PGFL using inverse frequency from training data
        alpha_init_pgfl = num_majority_train / (num_majority_train + num_minority_train)

        # Parameters for GAN Loss
        GAN_LAMBDA = 0.1 # Weighting factor for the adversarial loss component in the generator's total loss
        # For the GAN, we'll use BCE as the base classification loss for the generator

        # Parameters for Meta-Learning
        META_LEARNING_RATE = 0.01 # Learning rate for the meta-optimizer
        INNER_EPOCHS = 1 # Number of inner loop epochs per outer loop epoch
        # The base loss function for the inner loop (model training) and meta-validation loss
        BASE_CLASSIFICATION_LOSS_FN = tf.keras.losses.BinaryCrossentropy(from_logits=True)


        # Calculate input_dim from the scaled training data shape
        input_dim = X_train_scaled.shape[1]
        print(f"Input dimension for the model: {input_dim}")

        loss_results_for_file = [] # List to store results for each loss function for the current file

        for loss_type in LOSS_TYPES_TO_COMPARE:
            print(f"\n--------------------------------------------------------")
            print(f"Training {current_file_name} with {loss_type} Loss")
            print(f"--------------------------------------------------------")

            # Re-initialize the model(s) and optimizers for each loss function to ensure a fresh start
            model = SimpleMLP_TF(input_dim)
            optimizer = tf.keras.optimizers.Adam(learning_rate=0.001) # Optimizer for the main model (Generator)

            discriminator_model = None # Discriminator is only needed for GAN loss
            discriminator_optimizer = None # Discriminator optimizer is only needed for GAN loss

            minority_loss_weight = None # Trainable weight for Meta-Learning
            meta_optimizer = None # Optimizer for Meta-Learning weight

            loss_function = None # Classification loss function (used directly or as base for GAN/Meta-Learning)
            history = None # Initialize history variable


            if loss_type == 'AMRL':
                print("Margin Loss Parameters (Initial Values):")
                print(f"  BCE Weights: w1_bce (minority) = {w1_bce_amrl:.4f}, w0_bce (majority) = {w0_bce_amrl:.4f}")
                print(f"  Margins: m1 (pos) = {M_POS_INIT}, m0 (neg) = {M_NEG_INIT}")
                print(f"  Lambdas: lambda1 (pos) = {LAMBDA_POS_INIT}, lambda0 (neg) = {LAMBDA_NEG_INIT}")
                print(f"  Betas: beta1 (pos) = {BETA_POS_INIT}, beta0 = {BETA_NEG_INIT}")
                loss_function = AMRLLossTF(w1 = w1_bce_amrl, w0 = w0_bce_amrl, m1_init = M_POS_INIT, m0_init = M_NEG_INIT,
                                          lambda1_init=LAMBDA_POS_INIT, lambda0_init = LAMBDA_NEG_INIT,
                                          beta1 = BETA_POS_INIT, beta0 = BETA_NEG_INIT)

                print("\nStarting custom training loop for AMRL...")
                early_stopping_callback = EarlyStopping(monitor='val_loss', patience=EARLY_STOPPING_PATIENCE, min_delta=MIN_DELTA, restore_best_weights=True)
                history = custom_train_loop(model, loss_function, optimizer, train_dataset_tf, val_dataset_inner, NUM_EPOCHS, early_stopping_callback, CLIP_NORM) # Use val_dataset_inner for validation
                print("Custom training loop finished.")

            elif loss_type == 'PGFL':
                print("PGFL Loss Parameters (Initial Values):")
                print(f"  Gamma: {gamma_pgfl:.4f}")
                print(f"  Initial Alpha (Minority Weight): {alpha_init_pgfl:.4f}")
                loss_function = PGFLossTF(num_majority=num_majority_train, num_minority=num_minority_train,
                                          gamma=gamma_pgfl, alpha_init=alpha_init_pgfl)


                print("\nStarting custom training loop for PGFL...")
                early_stopping_callback = EarlyStopping(monitor='val_loss', patience=EARLY_STOPPING_PATIENCE, min_delta=MIN_DELTA, restore_best_weights=True)
                history = custom_train_loop_pgfl(model, loss_function, optimizer, train_dataset_tf, val_dataset_inner, NUM_EPOCHS, early_stopping_callback, CLIP_NORM) # Use val_dataset_inner for validation
                print("Custom training loop finished.")

            elif loss_type == 'GAN':
                print("GAN Loss Parameters:")
                print(f"  GAN Lambda (Adversarial Loss Weight): {GAN_LAMBDA:.4f}")
                classification_loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True)

                input_dim_features = X_train_scaled.shape[1]
                discriminator_model = Discriminator(input_dim_X=input_dim_features, input_dim_Y=1)
                discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

                print("\nStarting custom training loop for GAN...")
                early_stopping_callback = EarlyStopping(monitor='val_loss', patience=EARLY_STOPPING_PATIENCE, min_delta=MIN_DELTA, restore_best_weights=True)
                history = custom_train_loop_gan(model, discriminator_model, optimizer, discriminator_optimizer,
                                                classification_loss_fn, train_dataset_tf, val_dataset_inner, # Use val_dataset_inner for validation
                                                NUM_EPOCHS, early_stopping_callback, CLIP_NORM, GAN_LAMBDA)
                print("Custom training loop finished.")



            else: # Use model.compile and model.fit for other loss functions (BCE, FOCAL, CB)
                metrics_list = [
                    BinaryAccuracy(name = 'accuracy', threshold = 0.5),
                    Precision(name = 'precision', thresholds = 0.5),
                    Recall(name = 'recall', thresholds = 0.5),
                    AUC(name = 'auc_roc'),
                    AUC(curve = 'PR', name ='auc_pr')
                ]

                if loss_type == 'BCE':
                    print("Standard Binary Crossentropy Loss")
                    loss_function = tf.keras.losses.BinaryCrossentropy(from_logits = True)
                elif loss_type == 'FOCAL':
                    print(f"Focal Loss Parameters: alpha={alpha_focal:.4f}, gamma={gamma_focal:.4f}")
                    loss_function = FocalLossTF(alpha = alpha_focal, gamma = gamma_focal)
                elif loss_type == 'CB':
                    print(f"Class Balanced Loss Parameters: beta={beta_cb:.4f}")
                    loss_function = ClassBalancedLossTF(num_majority = num_majority_train, num_minority = num_minority_train, beta = beta_cb)
                else:
                    raise ValueError(f"Invalid LOSS_TYPE: {loss_type}. Choose from 'AMRL', 'BCE', 'FOCAL', 'CB', 'PGFL', 'GAN'.")


                # Compile the model using the selected loss function and metrics
                model.compile(optimizer=optimizer, # Use the same optimizer
                              loss=loss_function,
                              metrics=metrics_list)

                # Define Early Stopping callback for model.fit
                early_stopping_callback = EarlyStopping(monitor='val_loss',
                                               patience=EARLY_STOPPING_PATIENCE,
                                               min_delta=MIN_DELTA,
                                               restore_best_weights=True,
                                               verbose=1)

                print("\nStarting model.fit training...")
                history = model.fit(X_train_scaled, y_train_tf,
                                    epochs=NUM_EPOCHS,
                                    batch_size=BATCH_SIZE,
                                    validation_data=(X_val_scaled, y_val_tf), # Use the original val set for model.fit
                                    callbacks=[early_stopping_callback],
                                    verbose=1)
                history = history.history

                print("model.fit training finished.")

            # --- Plotting Training History ---
            if history: # Check if history is not None
                print("\nPlotting training history...")
                if loss_type == 'GAN':
                     # Plot GAN specific losses
                    gan_history_to_plot = {k: history[k] for k in ['gen_loss', 'disc_loss', 'gen_clf_loss', 'gen_adv_loss'] if k in history}
                    plot_training_history(gan_history_to_plot, current_file_name, f'{loss_type}_Losses', plots_save_dir)

                    # Plot Generator's classification metrics (using 'accuracy', etc keys which are for generator)
                    gen_metric_history_to_plot = {k: history[k] for k in ['accuracy', 'precision', 'recall', 'auc_roc', 'auc_pr',
                                                                      'val_accuracy', 'val_precision', 'val_recall', 'val_auc_roc', 'val_auc_pr',
                                                                      'val_loss'] if k in history
                                                                      }
                    # Rename 'val_loss' to 'val_gen_clf_loss' for clarity in plot title/labels if it exists
                    if 'val_loss' in gen_metric_history_to_plot:
                        gen_metric_history_to_plot['val_gen_clf_loss'] = gen_metric_history_to_plot.pop('val_loss')

                    plot_training_history(gen_metric_history_to_plot, current_file_name, f'{loss_type}_Generator_Metrics', plots_save_dir)


                else:
                     # For other losses, plot standard metrics including validation loss
                    plot_history = {k: history[k] for k in history if k in ['loss', 'accuracy', 'precision', 'recall', 'auc_roc', 'auc_pr',
                                                                            'val_loss', 'val_accuracy', 'val_precision', 'val_recall', 'val_auc_roc', 'val_auc_pr']}
                    plot_training_history(plot_history, current_file_name, loss_type, plots_save_dir)


            print("\nEvaluating on the Test Set...")

            print("Train class distribution:", np.bincount(y_train.values.flatten()))
            print("Val class distribution:", np.bincount(y_val.values.flatten()))
            print("Test class distribution:", np.bincount(y_test.values.flatten()))


            # Evaluate the model using model.evaluate or manual calculation for custom loop
            # Calculate metrics manually using predictions for all loss types for consistency
            y_pred_logits_test = model.predict(X_test_scaled, batch_size=BATCH_SIZE)
            y_pred_probs_test = tf.sigmoid(y_pred_logits_test).numpy().flatten()
            y_pred_binary_test = (y_pred_probs_test > 0.5).astype(int)
            y_true_test_flat = y_test.values.flatten()

            # 1. Get predicted probabilities from model
            y_pred_probs = model.predict(X_test_scaled, batch_size=BATCH_SIZE)

            # 2. Convert probabilities to class predictions (threshold = 0.5)
            y_pred_labels = (y_pred_probs >= 0.5).astype(int)

            # 3. Ensure y_test is in correct format (binary integers)
            y_true = np.array(y_test_tf).astype(int)

            # 4. Calculate metrics
            accuracy = round(accuracy_score(y_true, y_pred_labels),2)
            precision = round(precision_score(y_true, y_pred_labels, zero_division=0),2)
            recall = round(recall_score(y_true, y_pred_labels, zero_division=0),2)
            auc_roc = round(roc_auc_score(y_true, y_pred_probs),2)
            auc_pr = round(average_precision_score(y_true, y_pred_probs),2)

            # Confusion Matrix components
            tn, fp, fn, tp = confusion_matrix(y_true_test_flat, y_pred_binary_test, labels=[0, 1]).ravel()

            # Specificity (True Negative Rate)
            specificity = tn / (tn + fp) if (tn + fp) > 0 else 0.0

            # Geometric Mean (G-mean)
            current_recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
            specificity_gmean = tn / (tn + fp) if (tn + fp) > 0 else 0.0
            g_mean = np.sqrt(current_recall * specificity_gmean) if current_recall >= 0 and specificity_gmean >= 0 else np.nan
            g_mean = round(g_mean, 2)


            # Balanced Accuracy
            bal_accuracy = balanced_accuracy_score(y_true_test_flat, y_pred_binary_test)
            bal_accuracy = round(bal_accuracy, 2)

            # Kuipers Skill Score (KSS) / True Skill Score (TSS)
            kss_tss = current_recall + specificity - 1
            kss_tss = round(kss_tss, 2)

            # Heidke Skill Score (HSS)
            far = false_alarm(tp,tn,fp,fn)
            bias = bias_score(tp,tn,fp,fn)
            pod = probability_of_detection(tp,tn,fp,fn)
            hss= heidke_skill_score(tp,tn,fp,fn)
            pss = pierce_skill_score(tp,tn,fp,fn)
            f1  =  f1_score(tp,tn,fp,fn)

            num_minority_test = np.sum(y_test)
            num_majority_test = len(y_test) - num_minority_test

            #Combine Keras metrics and manually calculated metrics
            test_metrics_dict = {
                'Dataset': current_file_name,
                'Loss_Type': loss_type,
                #'Loss': loss, # Loss is not directly comparable across loss functions on the test set
                'Epochs': len(history['loss']) if loss_type not in ['GAN'] else (len(history['gen_loss']) if 'gen_loss' in history else 0), # Get epochs from appropriate history, handle empty history
                'Batch_Size': BATCH_SIZE,
                'Positive': num_minority_test,
                'Negative': num_majority_test,
                'TP': tp,
                'TN': tn,
                'FP': fp,
                'FN': fn,
                'Accuracy' : accuracy,
                'Precision': precision,
                'Recall': recall,
                'AUC-ROC': auc_roc,
                'AUC-PR': auc_pr,
                'Specificity': specificity,
                'G-Mean': g_mean,
                'Balanced_Accuracy': bal_accuracy,
                'KSS_TSS': kss_tss,
                'HSS': hss,
                'PSS': pss,
                'FAR': far,
                'bias': bias,
                'pod': pod,
                'f1': f1
            }


            # If the loss type is AMRL, add the learned parameters to the results dictionary
            if loss_type == 'AMRL':
                 if isinstance(loss_function, AMRLLossTF):
                    test_metrics_dict['Learned_M_POS'] = loss_function.m1.numpy()
                    test_metrics_dict['Learned_M_NEG'] = loss_function.m0.numpy()
                    test_metrics_dict['Learned_LAMBDA_POS'] = loss_function.lambda1.numpy()
                    test_metrics_dict['Learned_LAMBDA_NEG'] = loss_function.lambda0.numpy()

            # If the loss type is PGFL, add the learned alpha and final moving averages
            elif loss_type == 'PGFL':
                 if isinstance(loss_function, PGFLossTF):
                    test_metrics_dict['Learned_Alpha'] = loss_function.alpha.numpy()
                    test_metrics_dict['Final_MC0'] = loss_function.mc0.numpy()
                    test_metrics_dict['Final_MC1'] = loss_function.mc1.numpy()
            # If the loss type is GAN, add the GAN_LAMBDA parameter
            elif loss_type == 'GAN':
                 test_metrics_dict['GAN_Lambda'] = GAN_LAMBDA


            for metric, value in test_metrics_dict.items():
                if isinstance(value, float):
                    print(f"{metric}: {value:.4f}")
                else:
                    print(f"{metric}: {value}")

            # --- Consolidate and Save All Results ---
            # Save this loss result in the list
            loss_results_for_file.append(test_metrics_dict.copy())

        # --- Save all loss results for the current file ---
        file_results_df = pd.DataFrame(loss_results_for_file)
        output_filename = f"{current_file_name}_all_losses.xlsx"
        results_save_dir = os.path.join(GOOGLE_DRIVE_PATH, 'text files')
        os.makedirs(results_save_dir, exist_ok=True)
        print(f"Results will be saved to: {results_save_dir}")
        output_filepath = os.path.join(results_save_dir, output_filename) # Use GOOGLE_DRIVE_PATH for saving
        file_results_df.to_excel(output_filepath, index=False)
        # output_filepath = os.path.join(GOOGLE_DRIVE_PATH, output_filename)
        # file_results_df.to_csv(output_filepath, index=False)

        print(f"\nResults for {current_file_name} saved to: {output_filepath}")
        print("\nComprehensive Comparison Summary for this file:")
        print(file_results_df.round(4).to_string())

Scanning for XLSX files in: /content/drive/MyDrive/1. Trial/
Plots will be saved to: /content/drive/MyDrive/1. Trial/training_plots
Results will be saved to: /content/drive/MyDrive/1. Trial/text files

############################################################
Processing Dataset: 206_output_final_modified_less_columns.xlsx
############################################################
DataFrame shape after initial load: (4713, 20)
        Date  Station_Code  Maximum_Temperature  Minimum_Temperature  \
0 1999-11-01           NaN                  NaN                 -5.0   
1 1999-11-02           NaN                 16.5                 -7.0   
2 1999-11-03           NaN                 15.5                 -5.0   
3 1999-11-04           NaN                 15.5                  0.0   
4 1999-11-05           NaN                  2.0                 -1.0   

   Average_Temperature  Dry_Temperature  Snow_Temperature  \
0                  NaN             -3.0               NaN   
1         

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



Epoch 1/10
Train Loss: 1.0562, Train Acc: 0.7925, Train Prec: 0.4396, Train Recall: 0.3272, Train AUC-ROC: 0.7382, Train AUC-PR: 0.4069
Val Loss: 0.5532, Val Acc: 0.7485, Val Prec: 0.3958, Val Recall: 0.6129, Val AUC-ROC: 0.7955, Val AUC-PR: 0.5142
  AMRL Learned Params: m1=0.6393, m0=-0.6754, lambda1=1.6519, lambda0=0.7145

Epoch 2/10
Train Loss: 0.4657, Train Acc: 0.8004, Train Prec: 0.4733, Train Recall: 0.4286, Train AUC-ROC: 0.7671, Train AUC-PR: 0.4622
Val Loss: 0.2946, Val Acc: 0.7853, Val Prec: 0.4524, Val Recall: 0.6129, Val AUC-ROC: 0.8019, Val AUC-PR: 0.5227
  AMRL Learned Params: m1=0.2870, m0=-0.3307, lambda1=1.3776, lambda0=0.3872

Epoch 3/10
Train Loss: 0.2417, Train Acc: 0.7741, Train Prec: 0.4268, Train Recall: 0.5438, Train AUC-ROC: 0.7856, Train AUC-PR: 0.4664
Val Loss: 0.1992, Val Acc: 0.6810, Val Prec: 0.3478, Val Recall: 0.7742, Val AUC-ROC: 0.8173, Val AUC-PR: 0.5471
  AMRL Learned Params: m1=0.0593, m0=-0.1185, lambda1=1.2464, lambda0=0.0444

Epoch 4/10
Train L

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


456/456 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8116 - auc_pr: 0.2547 - auc_roc: 0.5447 - loss: 0.4753 - precision: 0.3854 - recall: 0.0339 - val_accuracy: 0.8160 - val_auc_pr: 0.3842 - val_auc_roc: 0.6213 - val_loss: 0.4268 - val_precision: 0.6250 - val_recall: 0.0806
Epoch 2/10
456/456 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8032 - auc_pr: 0.3273 - auc_roc: 0.5755 - loss: 0.4495 - precision: 0.6340 - recall: 0.0766 - val_accuracy: 0.8098 - val_auc_pr: 0.2675 - val_auc_roc: 0.5504 - val_loss: 0.4225 - val_precision: 0.5000 - val_recall: 0.0323
Epoch 3/10
456/456 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8066 - auc_pr: 0.3137 - auc_roc: 0.5740 - loss: 0.4087 - precision: 0.5567 - recall: 0.0459 - val_accuracy: 0.8160 - val_auc_pr: 0.3507 - val_auc_roc: 0.5952 - val_loss: 0.4120 - val_precision: 0.6667 - val_recall: 0.0645
Epoch 4/10
456/456 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8115 - auc_pr: 0.3091 - auc_roc: 0.5739 - loss: 0.4117 - precision: 0.5510 - reca

...

456/456 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7994 - auc_pr: 0.2987 - auc_roc: 0.5925 - loss: 9.2767e-04 - precision: 0.4097 - recall: 0.1378 - val_accuracy: 0.8374 - val_auc_pr: 0.4335 - val_auc_roc: 0.6837 - val_loss: 8.2246e-04 - val_precision: 0.6800 - val_recall: 0.2742
Epoch 2/10
456/456 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8125 - auc_pr: 0.3701 - auc_roc: 0.6578 - loss: 8.1622e-04 - precision: 0.4927 - recall: 0.2239 - val_accuracy: 0.8313 - val_auc_pr: 0.4701 - val_auc_roc: 0.7057 - val_loss: 8.0783e-04 - val_precision: 0.6400 - val_recall: 0.2581
Epoch 3/10
456/456 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8241 - auc_pr: 0.3642 - auc_roc: 0.6783 - loss: 7.8615e-04 - precision: 0.4962 - recall: 0.2140 - val_accuracy: 0.8405 - val_auc_pr: 0.4650 - val_auc_roc: 0.7168 - val_loss: 7.9858e-04 - val_precision: 0.6250 - val_recall: 0.4032
Epoch 4/10
456/456 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7991 - auc_pr: 0.4008 - auc_roc: 0.6742 - loss: 8.5113e-0

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Train Loss: -5.2993, Train Acc: 0.3553, Train Prec: 0.2176, Train Recall: 0.9194, Train AUC-ROC: 0.5601, Train AUC-PR: 0.2067
Val Loss: -44.1994, Val Acc: 0.1902, Val Prec: 0.1902, Val Recall: 1.0000, Val AUC-ROC: 0.5000, Val AUC-PR: 0.1902
  PGFL Learned Alpha: 1.4167, MC0: 0.5511, MC1: 0.9825

Epoch 2/10
Train Loss: -575.3597, Train Acc: 0.1904, Train Prec: 0.1904, Train Recall: 1.0000, Train AUC-ROC: 0.5000, Train AUC-PR: 0.1904
Val Loss: -1627.6294, Val Acc: 0.1902, Val Prec: 0.1902, Val Recall: 1.0000, Val AUC-ROC: 0.5000, Val AUC-PR: 0.1902
  PGFL Learned Alpha: 1.9541, MC0: 0.5511, MC1: 1.0000

Epoch 3/10
Train Loss: -5070.6680, Train Acc: 0.1904, Train Prec: 0.1904, Train Recall: 1.0000, Train AUC-ROC: 0.5000, Train AUC-PR: 0.1904
Val Loss: -9917.6484, Val Acc: 0.1902, Val Prec: 0.1902, Val Recall: 1.0000, Val AUC-ROC: 0.5000, Val AUC-PR: 0.1902
  PGFL Learned Alpha: 2.4415, MC0: 0.5511, MC1: 1.0000

Epoch 4/10
Train Loss: -21094.1133, Train Acc: 0.1904, Train Prec: 0.1904, Tra

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


KeyError: 'binary_accuracy'

In [ ]:


# Assuming the results files are saved in the 'text files' directory within GOOGLE_DRIVE_PATH

drive.mount('/content/drive', force_remount=True)

GOOGLE_DRIVE_PATH = '/content/drive/MyDrive/1. Trial/'
results_save_dir = os.path.join(GOOGLE_DRIVE_PATH, 'text files')

# List all the result files in the directory
result_files = [f for f in os.listdir(results_save_dir) if f.endswith('_all_losses.xlsx')]

if not result_files:
    print(f"No result files found in {results_save_dir}. Please ensure the main execution block ran successfully.")
else:
    all_results_list = []
    for file_name in result_files:
        file_path = os.path.join(results_save_dir, file_name)
        try:
            df_results = pd.read_excel(file_path)
            all_results_list.append(df_results)
        except Exception as e:
            print(f"Error reading file {file_path}: {e}")

    if all_results_list:
        # Concatenate all results into a single DataFrame
        all_results_df = pd.concat(all_results_list, ignore_index=True)

        print("\n--- Performance Summary Across Datasets and Loss Functions ---")

        # Group by Loss_Type and calculate mean for key metrics
        # Select key metrics for summary
        summary_metrics = ['Accuracy', 'Precision', 'Recall', 'AUC-ROC', 'AUC-PR',
                           'Specificity', 'G-Mean', 'Balanced_Accuracy', 'KSS_TSS', 'HSS', 'PSS', 'FAR', 'bias', 'pod', 'f1']

        # Ensure only existing columns are included in summary_metrics
        summary_metrics = [metric for metric in summary_metrics if metric in all_results_df.columns]


        performance_summary = all_results_df.groupby('Loss_Type')[summary_metrics].mean().round(4)

        print("Average Performance Metrics per Loss Function (averaged across datasets):")
        display(performance_summary)

        # Optional: Also show metrics per dataset per loss function if needed
        # print("\nPerformance Metrics per Dataset per Loss Function:")
        # display(all_results_df[['Dataset', 'Loss_Type'] + summary_metrics].round(4))

    else:
        print("No data available to summarize after attempting to read result files.")

# Task
Implement a new loss function and training strategy based on meta-learning reweighting and fine-tuning for extreme samples, and integrate it into the existing code for comparison with other loss functions.

## Define the meta-learning reweighting component

### Subtask:
Implement a mechanism to learn the weights for the loss function using a meta-learning approach. This will likely involve an outer loop that optimizes the weighting parameters based on the model's performance on a small validation set or a specific metric, and an inner loop that trains the main model with the current weights.


**Reasoning**:
Define a separate small "meta-validation" dataset and implement a mechanism to define and update sample weights using trainable `tf.Variable`s.



In [ ]:
# Step 1: Define a separate small "meta-validation" dataset
# We can split the existing validation set further. Let's use 50% of the original validation set for meta-validation.
# The remaining 50% will be the standard validation set used in the inner loop.

# Assuming X_val_scaled and y_val_tf from previous steps are available
# Split the validation set into validation and meta-validation sets
X_val_inner, X_meta_val, y_val_inner, y_meta_val = train_test_split(
    X_val_scaled, y_val_tf, test_size=0.5, random_state=42, stratify=y_val_tf
)

# Create TensorFlow Datasets for inner validation and meta-validation
val_dataset_inner = tf.data.Dataset.from_tensor_slices((X_val_inner.astype(np.float32), y_val_inner)).batch(BATCH_SIZE)
meta_val_dataset_tf = tf.data.Dataset.from_tensor_slices((X_meta_val.astype(np.float32), y_meta_val)).batch(BATCH_SIZE)


# Step 2: Implement a mechanism to define and update sample weights
# We'll use trainable tf.Variables for sample weights.
# A simple approach is to have a single trainable scalar weight that gets applied to the loss
# of the minority class samples in the inner loop, or perhaps a weight per class.
# Let's start with a simple trainable scalar weight applied to the minority class loss.

# Initialize a trainable weight for the minority class.
# This weight will be learned by the meta-learning process.
# Start with a value of 1.0, meaning equal weighting initially (like standard BCE).
# We can also initialize it based on inverse class frequency or other heuristics.
# Let's initialize it to reflect the imbalance ratio slightly biased towards minority.
initial_minority_weight = float(num_majority_train / num_minority_train) # Using training set ratio for initial weight

# We can make this weight trainable. Let's call it `minority_loss_weight`.
# This weight will be updated in the outer loop.
minority_loss_weight = tf.Variable(initial_minority_weight, dtype=tf.float32, trainable=True, name="minority_loss_weight")

print(f"Initial minority loss weight: {minority_loss_weight.numpy():.4f}")

# We also need a way to apply this weight within the inner loop's loss calculation.
# This will require modifying the base classification loss function or the custom train step.
# Let's plan to apply this weight in the inner training step.
